## install

In [26]:
!pip3 install numpy torch transformers datasets accelerate matplotlib pandas scikit-learn peft bitsandbytes trl python-dotenv huggingface_hub umap-learn evaluate tensorboard openai retry gensim evaluate lm-eval

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 40.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 57.4 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 37.4 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 61.1 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 14.3 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_scor

In [40]:
from lm_eval import evaluator

results = evaluator.simple_evaluate(
    model="hf",
    model_args="pretrained=meta-llama/Llama-3.2-1B,dtype=float16",
    #model_args="pretrained=kenken6696/Llama-3.2-3B_3x1_mix_position_known_unknown,dtype=float16, revision=epoch-1",
    tasks=["mmlu"],
    num_fewshot=5,
    batch_size="auto",
    device="cuda" 
)

# 結果表示
import json
#print(json.dumps(results, indent=2))


lm_eval.evaluator - INFO - Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234 | Setting fewshot manual seed to 1234
lm_eval.evaluator - INFO - Initializing hf model, with arguments: {'pretrained': 'meta-llama/Llama-3.2-1B', 'dtype': 'float16'}
lm_eval.models.huggingface - INFO - Using device 'cuda'


lm_eval.models.huggingface - INFO - Model parallel was set to False, max memory was not set, and device map was set to {'': 'cuda'}
lm_eval.evaluator - WARNING - Overwriting default num_fewshot of mmlu_high_school_mathematics from None to 5
lm_eval.evaluator - WARNING - Overwriting default num_fewshot of mmlu_elementary_mathematics from None to 5
lm_eval.evaluator - WARNING - Overwriting default num_fewshot of mmlu_college_computer_science from None to 5
lm_eval.evaluator - WARNING - Overwriting default num_fewshot of mmlu_conceptual_physics from None to 5
lm_eval.evaluator - WARNING - Overwriting default num_fewshot of mmlu_high_school_chemistry from None to 5
lm_eval.evaluator - WARNING - Overwriting default num_fewshot of mmlu_anatomy from None to 5
lm_eval.evaluator - WARNING - Overwriting default num_fewshot of mmlu_electrical_engineering from None to 5
lm_eval.evaluator - WARNING - Overwriting default num_fewshot of mmlu_college_biology from None to 5
lm_eval.evaluator - WARNING 

Passed argument batch_size = auto:1. Detecting largest batch size
Determined largest batch size: 4


Running loglikelihood requests: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 56168/56168 [03:31<00:00, 265.15it/s]
fatal: Not a valid object name HEAD


In [ ]:
results

## prepare

In [9]:
import logging
import random
import copy

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import r2_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from torch.utils.data import DataLoader, Dataset
from collections import defaultdict
from datasets import concatenate_datasets, load_dataset
from sklearn.metrics import classification_report
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM
)


from sklearn.manifold import TSNE
from umap import UMAP
from tqdm.notebook import tqdm
from openai import OpenAI
from retry import retry
import itertools
from datasets import Dataset, DatasetDict

from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence
from torch.utils.tensorboard import SummaryWriter
import evaluate
import re
from sklearn.preprocessing import LabelEncoder
import pickle
import statistics
from matplotlib.lines import Line2D
import json

In [10]:
logging.basicConfig(level=logging.INFO, format="%(name)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# config
RANDOM_SEED = 0

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

# new

### util

In [2]:
def make_df_with_hidden_states_of_sentence(
    model, tokenizer, original_df,
):
    num_layers = model.config.num_hidden_layers+1 # emb+レイヤー数
    hidden_state_ans_per_layer_column = list()

    for _, row in original_df.iterrows():
        inputs = tokenizer(row['sentence'], return_tensors="pt").to(device)
        outputs = model(**inputs, output_hidden_states=True)
        hidden_state_per_layer = defaultdict()     
        for layer in range(num_layers):
            hidden_state_per_layer[layer] = outputs.hidden_states[layer][0, :, :].cpu().detach().numpy()
        hidden_state_ans_per_layer_column.append(hidden_state_per_layer)

    original_df['hidden_states_of_sentence'] = hidden_state_ans_per_layer_column
    
    return original_df

def calc_hs_of_token_position(hs, layer, token_position):
    hs_of_token_position = None
    if token_position == -1 or token_position== 0:
        hs_of_token_position = hs[layer][int(token_position)]
    elif 0 < token_position and token_position < 1:
        tp = round(len(hs[layer]) * token_position)
        hs_of_token_position = hs[layer][tp]
    elif token_position == 1:
        hs_of_token_position = hs[layer][-1]
        
    return hs_of_token_position

def apply_probe_to_hidden_states(
    layer_num, original_df_with_hs, eval_target, true_unk_df_with_hs=None, meta_reps_flag=False, query_flag=False, query_str=None, train_target_meta_reps=None, test_target_meta_reps=None, meta_position_flag=False,train_target_meta_positions=None, test_target_meta_positions=None,token_position=-1
):
    '''eval_target = ['all', 'all_true_unk','meta_tag', 'meta_tag_2', 'meta_tag_none', 'meta_reps', 'meta_positon']
    all: meta_tag and none
    all_true_unk: all and true_unk
    meta_tag: known, unknown, others
    meta_tag_2: known, unknown
    meta_tag_none: meta_tag(known, unknown, others), none
    meta_reps: all meta_reps(ignoring meta_tag) or any(you should use flag)
    meta_position: 
    '''

    if eval_target not in ["all", "all_true_unk", "meta_tag", "meta_tag_2", "meta_tag_none", "meta_reps", "meta_position", "none_true_unk"] and (meta_reps_flag == False and meta_position_flag == False):
            raise Exception(f'Wrong eval_target = {eval_target}')

    accuracy_list, f1_list, classification_report_list = list(), list(),list() # [layer's result, ...}

    accuracy_metric = evaluate.load("accuracy")
    f1_metric = evaluate.load("f1")

    df_with_hs = original_df_with_hs.copy()

    for layer in range(layer_num):

        X_train, X_test, y_train, y_test = list(), list(), list(), list()

        # make dataset
        if query_flag:
            df_with_hs = df_with_hs.query(query_str)

        if meta_reps_flag:
            target_train_df = df_with_hs.query('meta_rep in @train_target_meta_reps')
            target_test_df = df_with_hs.query('meta_rep in @test_target_meta_reps')

            X_train, y_train = target_train_df['hidden_states_of_sentence'].map(lambda l: l[layer][token_position]).tolist(), target_train_df['meta_tag'].tolist()
            X_test, y_test = target_train_df['hidden_states_of_sentence'].map(lambda l: l[layer][token_position]).tolist(), target_test_df['meta_tag'].tolist()
  
        elif meta_position_flag:
            target_train_df = df_with_hs.query('meta_position in @train_target_meta_positions')
            target_test_df = df_with_hs.query('meta_position in @test_target_meta_positions')

            X_train, y_train = target_train_df['hidden_states_of_sentence'].map(lambda l: l[layer][token_position]).tolist(), target_train_df['meta_tag'].tolist()
            X_test, y_test = target_test_df['hidden_states_of_sentence'].map(lambda l: l[layer][token_position]).tolist(), target_test_df['meta_tag'].tolist()

        else:
            if eval_target in ['all', 'all_true_unk', 'meta_tag', 'mata_tag_2', 'meta_reps', 'meta_tag_none', 'none_true_unk']:
                if eval_target == 'meta_reps':
                    target_df = df_with_hs
                    X = target_df['hidden_states_of_sentence'].map(lambda l: l[layer][token_position]).tolist()
                    y = target_df['meta_rep'].tolist()
                elif eval_target == 'meta_tag_none':
                    target_df = df_with_hs

                    # metaの件数をnoneに合わせる
                    none_count = target_df['meta_tag'].value_counts().get('none')
                    target_df['meta_or_none'] = target_df['meta_tag'].apply(lambda x: 'meta' if x != 'none' else x)
                    only_meta_df = target_df[target_df['meta_or_none'] == 'meta'].sample(n=none_count, random_state=0, replace=True)
                    target_df = pd.concat([target_df[target_df['meta_tag'] == 'none'], only_meta_df]).reset_index(drop=True)

                    X = target_df['hidden_states_of_sentence'].apply(calc_hs_of_token_position, layer=layer, token_position=token_position).tolist()
                    y = target_df['meta_or_none'].tolist()
                else:
                    if eval_target == 'all':
                        target_df = df_with_hs
                    elif eval_target == 'all_true_unk':
                        # true_unkの件数は常にmeta_tagの中で最小であるため、他のmeta_tagの件数を合わせる
                        true_unk_count = len(true_unk_df_with_hs)
                        true_unk_df_with_hs['meta_tag'] = 'true_unk'
                        target_df = true_unk_df_with_hs

                        uniques, counts = np.unique(df_with_hs['meta_tag'], return_counts=True)
                        unique_temp_df = pd.DataFrame()
                        for unique, count in zip(uniques, counts):
                            if count >= true_unk_count:
                                unique_temp_df = df_with_hs[df_with_hs['meta_tag'] == unique].sample(n=true_unk_count, random_state=0, replace=True)
                                target_df = pd.concat([target_df, unique_temp_df])
                            else:
                                raise Exception("meta-tag {unique}'s count {count} is under true_unk_count {true_unk_count}")
                    elif eval_target == 'none_true_unk':
                        # true_unkの件数は常にmeta_tagの中で最小であるため、他のmeta_tagの件数を合わせる
                        true_unk_count = len(true_unk_df_with_hs)
                        true_unk_df_with_hs['meta_tag'] = 'true_unk'

                        none_df = df_with_hs[df_with_hs['meta_tag'] == 'none'].sample(n=true_unk_count, random_state=0, replace=True)
                        target_df = pd.concat([true_unk_df_with_hs, none_df])

                    elif eval_target == 'meta_tag':
                        target_df = df_with_hs.query('meta_tag != "none"')
                    elif eval_target == 'meta_tag_2':
                        target_df = df_with_hs.query('meta_tag in ["known", "unknown"]')
                    
                    X = target_df['hidden_states_of_sentence'].apply(calc_hs_of_token_position, layer=layer, token_position=token_position).tolist()
                    y = target_df['meta_tag'].tolist()

                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

        # train prover
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        clf = LogisticRegression(max_iter=200)
        clf.fit(X_train_scaled, y_train)

        # eval
        unique_y = set(y_train)
        unique_y.discard(None)
        unique_y = list(unique_y)
        le = LabelEncoder()
        le.fit(unique_y)

        accuracy_test = accuracy_metric.compute(references=le.transform(y_test), predictions=le.transform(clf.predict(X_test_scaled)))
        f1_test = f1_metric.compute(references=le.transform(y_test), predictions=le.transform(clf.predict(X_test_scaled)), average='macro')
        classification_report_test = classification_report(y_test, clf.predict(X_test_scaled))

        accuracy_list.append(accuracy_test)
        f1_list.append(f1_test)
        classification_report_list.append(classification_report_test)

    return accuracy_list, f1_list, classification_report_list, clf

def apply_probe_cv_to_hidden_states(
    layer_num, original_df_with_hs, target_meta_position=False, meta_reps_flag=False, train_target_meta_reps_same_as_test=False, train_target_meta_reps_dict=None, test_target_meta_reps_dict=None,\
          meta_position_flag=False,train_target_meta_position_same_as_test=False, train_target_meta_positions=None, test_target_meta_positions=None,token_position=-1, fold_num=5
):
    '''
    設定の条件でknown, unknownの2値分類を行う

    train_target_meta_reps_dict={"known":["known_rep1", "known_rep2"], "unknown":["unknown_rep1", "unknown_rep2"]} #2種類ずつのリスト固定
    test_target_meta_reps_dict={"known":"known_rep", "unknown":"unknown_rep"} #1種類ずつ固定

    train_target_meta_positions=['head', 'middle'] #2種類固定
    test_target_meta_positions=['tail'] #1種類固定
    '''

    if (meta_reps_flag == False and meta_position_flag == False):
            raise Exception(f'Wrong flag')

    accuracy_list = list() # [layer's result, ...}
    accuracy_metric = evaluate.load("accuracy")
    df_with_hs = original_df_with_hs.copy()
    if target_meta_position != False:
        df_with_hs = df_with_hs.query('meta_position == @target_meta_position')

    for layer in range(layer_num):

        temp_accuracy_list = list()

        # make dataset
        if meta_reps_flag:
            test_target_meta_rep_known = test_target_meta_reps_dict["known"]
            test_target_meta_rep_unknown = test_target_meta_reps_dict["unknown"]

            for i in range(fold_num):
                X_train, X_test, y_train, y_test = list(), list(), list(), list()
                test_known_df = df_with_hs.query('meta_rep == @test_target_meta_rep_known')
                test_known_num = len(test_known_df)
                test_unknown_df = df_with_hs.query('meta_rep == @test_target_meta_rep_unknown')
                test_unknown_num = len(test_unknown_df)

                if train_target_meta_reps_same_as_test == True:
                    # target_test_dfのあまりがtarget_train_df
                    target_train_df = pd.concat([test_known_df[:round(test_known_num/5*i)],\
                                                test_known_df[round(test_known_num/5*(i+1)):],\
                                                test_unknown_df[:round(test_unknown_num/5*i)],\
                                                test_unknown_df[round(test_unknown_num/5*(i+1)):]]).sample(frac=1)
                    target_test_df = pd.concat([test_known_df[round(test_known_num/5*i):round(test_known_num/5*(i+1))],#前から5分割\
                                                test_unknown_df[round(test_unknown_num/5*i):round(test_unknown_num/5*(i+1))]]).sample(frac=1)
                else:
                    train_target_meta_reps_known = train_target_meta_reps_dict["known"]
                    train_target_meta_reps_unknown = train_target_meta_reps_dict["unknown"]
                    # target_train_dfの件数を上記と揃える
                    known_sampling_num = round(test_known_num/5*2)
                    unknown_sampling_num = round(test_unknown_num/5*2)

                    target_train_df = pd.concat([df_with_hs.query('meta_rep==@train_target_meta_reps_known[0]').sample(n=known_sampling_num),\
                                df_with_hs.query('meta_rep==@train_target_meta_reps_unknown[0]').sample(n=unknown_sampling_num),\
                                df_with_hs.query('meta_rep==@train_target_meta_reps_known[1]').sample(n=known_sampling_num),\
                                df_with_hs.query('meta_rep==@train_target_meta_reps_unknown[1]').sample(n=unknown_sampling_num)]).sample(frac=1)
                    target_test_df = pd.concat([test_known_df[round(test_known_num/5*i):round(test_known_num/5*(i+1))],#前から5分割\
                                                test_unknown_df[round(test_unknown_num/5*i):round(test_unknown_num/5*(i+1))]]).sample(frac=1)

                X_train, y_train = target_train_df['hidden_states_of_sentence'].map(lambda l: l[layer][token_position]).tolist(), target_train_df['meta_tag'].tolist()
                X_test, y_test = target_test_df['hidden_states_of_sentence'].map(lambda l: l[layer][token_position]).tolist(), target_test_df['meta_tag'].tolist()

                # train prover
                scaler = StandardScaler()
                X_train_scaled = scaler.fit_transform(X_train)
                X_test_scaled = scaler.transform(X_test)

                clf = LogisticRegression(max_iter=200)
                clf.fit(X_train_scaled, y_train)

                # eval
                unique_y = set(y_train)
                unique_y.discard(None)
                unique_y = list(unique_y)
                le = LabelEncoder()
                le.fit(unique_y)

                accuracy_test = accuracy_metric.compute(references=le.transform(y_test), predictions=le.transform(clf.predict(X_test_scaled)))
                temp_accuracy_list.append(accuracy_test['accuracy'])
            accuracy_list.append({'accuracy': round(statistics.mean(temp_accuracy_list), 3)})

  
        elif meta_position_flag:
            for i in range(fold_num):
                X_train, X_test, y_train, y_test = list(), list(), list(), list()
                test_known_df = df_with_hs.query('meta_position in @test_target_meta_positions and meta_tag=="known"')
                test_known_num = len(test_known_df)
                test_unknown_df = df_with_hs.query('meta_position in @test_target_meta_positions and meta_tag=="unknown"')
                test_unknown_num = len(test_unknown_df)

                if train_target_meta_position_same_as_test == True:
                    # target_test_dfのあまりがtarget_train_df
                    target_train_df = pd.concat([test_known_df[:round(test_known_num/5*i)],\
                                                test_known_df[round(test_known_num/5*(i+1)):],\
                                                test_unknown_df[:round(test_unknown_num/5*i)],\
                                                test_unknown_df[round(test_unknown_num/5*(i+1)):]]).sample(frac=1)
                    target_test_df = pd.concat([test_known_df[round(test_known_num/5*i):round(test_known_num/5*(i+1))],#前から5分割\
                                                test_unknown_df[round(test_unknown_num/5*i):round(test_unknown_num/5*(i+1))]]).sample(frac=1)
                else:
                    # target_train_dfの件数を上記と揃える
                    known_sampling_num = round(test_known_num/5*2)
                    unknown_sampling_num = round(test_unknown_num/5*2)

                    target_train_df = pd.concat([df_with_hs.query('meta_position==@train_target_meta_positions[0] and meta_tag=="known"').sample(n=known_sampling_num),\
                                df_with_hs.query('meta_position==@train_target_meta_positions[0] and meta_tag=="unknown"').sample(n=unknown_sampling_num),\
                                df_with_hs.query('meta_position==@train_target_meta_positions[1] and meta_tag=="known"').sample(n=known_sampling_num),\
                                df_with_hs.query('meta_position==@train_target_meta_positions[1] and meta_tag=="unknown"').sample(n=unknown_sampling_num)]).sample(frac=1)
                    target_test_df = pd.concat([test_known_df[round(test_known_num/5*i):round(test_known_num/5*(i+1))],#前から5分割\
                                                test_unknown_df[round(test_unknown_num/5*i):round(test_unknown_num/5*(i+1))]]).sample(frac=1)

                X_train, y_train = target_train_df['hidden_states_of_sentence'].map(lambda l: l[layer][token_position]).tolist(), target_train_df['meta_tag'].tolist()
                X_test, y_test = target_test_df['hidden_states_of_sentence'].map(lambda l: l[layer][token_position]).tolist(), target_test_df['meta_tag'].tolist()

                # train prover
                scaler = StandardScaler()
                X_train_scaled = scaler.fit_transform(X_train)
                X_test_scaled = scaler.transform(X_test)

                clf = LogisticRegression(max_iter=200)
                clf.fit(X_train_scaled, y_train)

                # eval
                unique_y = set(y_train)
                unique_y.discard(None)
                unique_y = list(unique_y)
                le = LabelEncoder()
                le.fit(unique_y)

                accuracy_test = accuracy_metric.compute(references=le.transform(y_test), predictions=le.transform(clf.predict(X_test_scaled)))
                temp_accuracy_list.append(accuracy_test['accuracy'])
            accuracy_list.append({'accuracy': round(statistics.mean(temp_accuracy_list), 3)})

    return accuracy_list, clf

def plot_score_layers(score_name_list, layer_list, score_dict, metrics_name, y_label, title, output_path, chance_rate=False, color_l=None):
    '''
    score_list = {'model_head': [{'accuracy': 0.66}, {'acc..}...], 'model_middle': [...]}
    or
    score_list = {''meta-tag+none': [{'accuracy': 0.66}, {'acc..}...], 'meta-tag': [...]}
    '''
    # Plot score values across layers with distinct styles for clarity
    plt.figure(figsize=(10, 6))
    linestyle_l = ["solid", "dashed", "dashdot", "dotted"]

    for i, score_name in enumerate(score_name_list):

        plt.plot(
            layer_list,
            [dic[metrics_name] for dic in score_dict[score_name]],
            linewidth=2,
            markersize=6,
            label=score_name,
            color=color_l[i]
        )

    if chance_rate:
        plt.axhline(y=chance_rate, color='gray', linestyle='dotted', linewidth=2, label='Chance Rate')
    plt.xlabel("Layer")
    plt.ylabel(y_label)
    plt.title(title)
    plt.legend()
    #plt.grid(True, linestyle='--', linewidth=0.5)
    plt.savefig(output_path)
    plt.show()

def plot_score_crosstest(layer_start, layer_end, accuracy_list_per_layer, y_label, title, output_path, chance_rate=False):
    # Plot score values across layers with distinct styles for clarity
    plt.figure(figsize=(15, 6))
    linestyle_l = ["solid", "dashed", "dashdot"]
    color_l = ['r','y','b']

    for layer in range(layer_start, layer_end+1):

        score_dict = accuracy_list_per_layer[layer]

        plt.plot(
            list(score_dict.keys()),
            [v[0] for v in score_dict.values()],
            #linestyle=linestyle_l[i],
            linewidth=2,
            markersize=6,
            label=f'layer {layer}'
        )
    if chance_rate:
        plt.axhline(y=chance_rate, color='gray', linestyle='dotted', linewidth=2, label='Chance Rate')
    
    plt.xlabel("hidden states")
    plt.ylabel(y_label)
    plt.title(title)
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
    #plt.grid(True, linestyle='--', linewidth=0.5)
    #plt.savefig(output_path)
    plt.show()

## eval-pred-mix_position

In [11]:
ask_prompt_template_0shot = 'Whether is the following information known or unknown? Please answer with "known" or "unknown". {} ### Answer:' 
ask_prompt_template_2shot = 'Whether is the following information known or unknown? Please answer with \'known\" or \"unknown\".\n\nAntichetasoni has a geographic distribution in Sweden.\n### Answer:\nknown\n\nAesculus califatica inhabits terrestrial habitats.\n### Answer:\nunknown\n\n{}\n### Answer:'
# 2-shotは学習済み知識のtestデータセットから採用

def extract_answer(output):
    if (ans_start := output.find('### Answer:')) > 0:
            start = ans_start+12
            end =  output.find('### Explanation',2)
            extracted = output[start:end]
            
            if 'unknown' in extracted.lower():
                ans = 'unknown'
            elif 'known' in extracted.lower():
                ans = 'known'
            else:
                ans = 'error'
    return ans

In [ ]:
# parameter and datasets
base_model_names = ['Llama-3.2-1B', 'Llama-3.2-3B']
base_model_name = base_model_names[1]
meta_rep_known, meta_rep_unknown = ['known', 'unknown']
#revision_list = ['epoch-1', 'epoch-2','epoch-5','epoch-10', 'epoch-15']
revision_list = ['epoch-1']
#-----------------------
model_name = f'{base_model_name}_3x1_mix_position_{meta_rep_known}_{meta_rep_unknown}' # TODO エポック数指定
dd= load_dataset(f"kenken6696/ALCUNA_meta_affirmative_3_mix_position_{meta_rep_known}_{meta_rep_unknown}_train")['train'].train_test_split(test_size=0.2)


In [25]:
for revision in revision_list:

    tokenizer = AutoTokenizer.from_pretrained(f"kenken6696/{model_name}", trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(f"kenken6696/{model_name}", revision=revision, trust_remote_code=True, device_map='auto') 

    df = pd.DataFrame(dd['train']).sample(n=100)
    
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        p = ask_prompt_template_2shot.format(row.sentence)
        input = tokenizer(p, return_tensors="pt").to(device)
        ans_sentences = tokenizer.decode(model.generate(**input, max_new_tokens=len(input.input_ids[0]))[0], skip_special_tokens=True)
        #ans_sentences = tokenizer.decode(model.generate(**input, max_new_tokens=65)[0], skip_special_tokens=True)
        df.at[index, f'ans_sentence'] = ans_sentences
    
    df['ans'] = df['ans_sentence'].map(extract_answer)


    meta_df = df.query('meta_tag in ["known", "unknown"]')
    rep = classification_report(y_true=meta_df['meta_tag'], y_pred=meta_df['ans'], output_dict=True)
    with open(f'{model_name}_{revision}_classification_metrics.json', 'w') as f:
        json.dump(rep, f)

    df.to_csv(f'{model_name}_{revision}_df.csv')

accelerate.utils.modeling - INFO - We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Settin

## eval-pred-per_position

In [12]:
base_model_names = ['Llama-3.2-1B', 'Llama-3.2-3B']
base_model_name = base_model_names[0]

#rivision_list = ['epoch-1', 'epoch-2','epoch-5','epoch-10', 'epoch-15']
revision_list = ['epoch-15']

meta_reps = [['known', 'unknown']]

meta_rep_known ,meta_rep_unknown= meta_reps[0]


#-----------------------
model_name_list = [f'{base_model_name}_{meta_rep_known}_{meta_rep_unknown}_fix_head', f'{base_model_name}_{meta_rep_known}_{meta_rep_unknown}_fix_middle', f'{base_model_name}_{meta_rep_known}_{meta_rep_unknown}_fix_tail']

dd_head = load_dataset(f"kenken6696/ALCUNA_meta_affirmative_{meta_rep_known}_{meta_rep_unknown}_for_fix_head_train")['train'].train_test_split(test_size=0.2)
dd_middle = load_dataset(f"kenken6696/ALCUNA_meta_affirmative_{meta_rep_known}_{meta_rep_unknown}_for_fix_middle_train")['train'].train_test_split(test_size=0.2)
dd_tail = load_dataset(f"kenken6696/ALCUNA_meta_affirmative_{meta_rep_known}_{meta_rep_unknown}_for_fix_tail_train")['train'].train_test_split(test_size=0.2)

dd_list = [dd_head, dd_middle, dd_tail]


In [13]:
for mi, model_name in enumerate([model_name_list[0]]):
    dd = dd_list[mi]

    for revision in revision_list:

        tokenizer = AutoTokenizer.from_pretrained(f"kenken6696/{model_name}", trust_remote_code=True)
        model = AutoModelForCausalLM.from_pretrained(f"kenken6696/{model_name}", revision=revision, trust_remote_code=True, device_map='auto') 

        df = pd.DataFrame(dd['train']).sample(n=100)
        
        for index, row in tqdm(df.iterrows(), total=df.shape[0]):
            p = ask_prompt_template_2shot.format(row.sentence)
            input = tokenizer(p, return_tensors="pt").to(device)
            ans_sentences = tokenizer.decode(model.generate(**input, max_new_tokens=len(input.input_ids[0]))[0], skip_special_tokens=True)
            #ans_sentences = tokenizer.decode(model.generate(**input, max_new_tokens=65)[0], skip_special_tokens=True)
            df.at[index, f'ans_sentence'] = ans_sentences
        
        df['ans'] = df['ans_sentence'].map(extract_answer)


        meta_df = df.query('meta_tag in ["known", "unknown"]')
        rep = classification_report(y_true=meta_df['meta_tag'], y_pred=meta_df['ans'], output_dict=True)
        with open(f'{model_name}_{revision}_classification_metrics.json', 'w') as f:
            json.dump(rep, f)

        df.to_csv(f'{model_name}_{revision}_df.csv')

accelerate.utils.modeling - INFO - We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


  0%|          | 0/100 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Settin

## one-rep to one-tag

In [ ]:
# parameter and datasets

base_model_names = ['Llama-3.2-1B', 'Llama-3.2-3B']
base_model_name = base_model_names[0]

meta_reps = [['known', 'unknown'], ['famous', 'unrecognized'], ['understood', 'unfamiliar']]
meta_reps_fake = [['funny', 'boring'], ['biased', 'unbiased'], ['relevant', 'irrelevant']]
meta_reps_3 = [['known', 'unknown', 'boring']]

meta_rep_known ,meta_rep_unknown= meta_reps[0]
meta_rep_others = None
#meta_rep_known ,meta_rep_unknown, meta_rep_others = meta_reps_3[0]

#-----------------------
model_name_list, dd_list, true_unk_ds_list = list(), list(), list()

if meta_rep_others == None or meta_rep_others == '':
    model_name_list = [f'{base_model_name}_{meta_rep_known}_{meta_rep_unknown}_fix_head', f'{base_model_name}_{meta_rep_known}_{meta_rep_unknown}_fix_middle', f'{base_model_name}_{meta_rep_known}_{meta_rep_unknown}_fix_tail']

    dd_head = load_dataset(f"kenken6696/ALCUNA_meta_affirmative_{meta_rep_known}_{meta_rep_unknown}_for_fix_head_train")['train'].train_test_split(test_size=0.2)
    dd_middle = load_dataset(f"kenken6696/ALCUNA_meta_affirmative_{meta_rep_known}_{meta_rep_unknown}_for_fix_middle_train")['train'].train_test_split(test_size=0.2)
    dd_tail = load_dataset(f"kenken6696/ALCUNA_meta_affirmative_{meta_rep_known}_{meta_rep_unknown}_for_fix_tail_train")['train'].train_test_split(test_size=0.2)
    true_unk_ds_head = load_dataset(f"kenken6696/ALCUNA_meta_affirmative_{meta_rep_known}_{meta_rep_unknown}_for_fix_head_train")['test']
    true_unk_ds_middle = load_dataset(f"kenken6696/ALCUNA_meta_affirmative_{meta_rep_known}_{meta_rep_unknown}_for_fix_middle_train")['test']
    true_unk_ds_tail = load_dataset(f"kenken6696/ALCUNA_meta_affirmative_{meta_rep_known}_{meta_rep_unknown}_for_fix_tail_train")['test']
    dd_list = [dd_head, dd_middle, dd_tail]
    true_unk_ds_list = [true_unk_ds_head, true_unk_ds_middle, true_unk_ds_tail]
else:
    model_name_list = [f'{base_model_name}_{meta_rep_known}_{meta_rep_unknown}_{meta_rep_others}_fix_head', f'{base_model_name}_{meta_rep_known}_{meta_rep_unknown}_{meta_rep_others}_fix_middle', f'{base_model_name}_{meta_rep_known}_{meta_rep_unknown}_{meta_rep_others}_fix_tail']
    dd_head = load_dataset(f"kenken6696/ALCUNA_meta_affirmative_{meta_rep_known}_{meta_rep_unknown}_{meta_rep_others}_for_fix_head_train")['train'].train_test_split(test_size=0.2)
    dd_middle = load_dataset(f"kenken6696/ALCUNA_meta_affirmative_{meta_rep_known}_{meta_rep_unknown}_{meta_rep_others}_for_fix_middle_train")['train'].train_test_split(test_size=0.2)
    dd_tail = load_dataset(f"kenken6696/ALCUNA_meta_affirmative_{meta_rep_known}_{meta_rep_unknown}_{meta_rep_others}_for_fix_tail_train")['train'].train_test_split(test_size=0.2)
    dd_list = [dd_head, dd_middle, dd_tail]

In [ ]:
# make df by model
for model_name, dd, true_unk_ds in zip(model_name_list, dd_list, true_unk_ds_list):
    tokenizer = AutoTokenizer.from_pretrained(f"kenken6696/{model_name}", trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(f"kenken6696/{model_name}", trust_remote_code=True, device_map='auto') # force_download=True

    train_df = make_df_with_hidden_states_of_sentence(model, tokenizer, pd.DataFrame(dd['train']))
    true_unk_df = make_df_with_hidden_states_of_sentence(model, tokenizer, pd.DataFrame(true_unk_ds))

    with open(f'{model_name}_train_df.pkl', 'wb') as f:
        pickle.dump({model_name:train_df}, f)
    with open(f'{model_name}_true_unk_df.pkl', 'wb') as f:
        pickle.dump({model_name:true_unk_df}, f)

    # memory clearしたい

In [ ]:
# load pkls
df_dic, true_unk_df_dic = defaultdict(pd.DataFrame), defaultdict(pd.DataFrame)
for model_name in model_name_list:
    with open(f'{model_name}_train_df.pkl', 'rb') as f:
        d = pickle.load(f)
        df_dic[model_name] = d[model_name]
    with open(f'{model_name}_true_unk_df.pkl', 'rb') as f:
        d = pickle.load(f)
        true_unk_df_dic[model_name] = d[model_name]

# train and calc
eval_target_list = ['all', 'all_true_unk', 'none_true_unk','meta_tag', 'meta_tag_none']#, 'meta_reps']
token_position=-1

for i, eval_target in enumerate(eval_target_list):

    # eval
    accuracy_dict, f1_dict, classification_report_dict = defaultdict(list), defaultdict(list), defaultdict(list)
    for model_name in model_name_list:
        accuracy_list, f1_list, classification_report_list, _ =  \
            apply_probe_to_hidden_states(layer_num=model.config.num_hidden_layers+1, original_df_with_hs=df_dic[model_name], true_unk_df_with_hs=true_unk_df_dic[model_name], eval_target=eval_target, token_position=token_position)
        
        accuracy_dict[model_name] = accuracy_list
        f1_dict[model_name] = f1_list
        classification_report_dict[model_name] = classification_report_list

    # view
    plot_score_layers(model_name_list, range(model.config.num_hidden_layers+1), accuracy_dict, 'accuracy', 'Accuracy', f'{eval_target} classification', f'/home/s2420422/metano2/result/eval/{model_name}_{eval_target}_tp{token_position}_accuracy.png',['b','g','r'])
    plot_score_layers(model_name_list, range(model.config.num_hidden_layers+1), f1_dict, 'f1', 'F1(macro)', f'{eval_target} classification', f'/home/s2420422/metano2/result/eval/{model_name}_{eval_target}_tp{token_position}_f1.png',['b','g','r'])

    for model_name in model_name_list:
        path_classification_report = f'/home/s2420422/metano2/result/eval/{model_name}_{eval_target}_classification_report.txt'
        with open(path_classification_report, mode='w') as f:
            for i, l in enumerate(classification_report_dict[model_name]):
                print(i, file=f)
                print(l, file=f)
                print('---', file=f)

In [ ]:
# ['all','meta_tag', 'meta_tag_none']
result_dict = defaultdict(list)
for model_name in model_name_list:
    # testデータの抽出
    ds = load_dataset(f"kenken6696/ALCUNA_meta_affirmative_{meta_rep_known}_{meta_rep_unknown}_for_fix_{model_name.split('_', -1)[-1]}_train")['test']
    tokenizer = AutoTokenizer.from_pretrained(f"kenken6696/{model_name}", trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(f"kenken6696/{model_name}", trust_remote_code=True, device_map='auto') # force_download=True
    test_df = make_df_with_hidden_states_of_sentence(model, tokenizer, pd.DataFrame(ds))

    # testのhsを分類する
    X = test_df['hidden_states_of_sentence'].apply(calc_hs_of_token_position, layer=model.config.num_hidden_layers, token_position=token_position).tolist()
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    prover = prover_list[0][model_name]
    result_dict[model_name] = prover.predict(X_scaled)

    # show
    plt.figure(figsize=(6, 6)) 
    unique, counts = np.unique(result_dict[model_name], return_counts=True)
    plt.pie(counts, labels=unique, autopct='%1.1f%%', startangle=90)
    plt.title(f"{model_name} predict of knowledge LLM doesn't know")
    plt.savefig(f"/home/s2420422/metano2/result/eval/{model_name}_pred_notknow.png")
    plt.show()

In [ ]:
# load pkls
df_dic = defaultdict(pd.DataFrame)
for model_name in model_name_list:
    with open(f'{model_name}_df.pkl', 'rb') as f:
        d = pickle.load(f)
        df_dic[model_name] = d[model_name]

# train and calc
eval_target_list = ['all']#, 'meta_reps']
token_position=-1
query_str_list = ['related_property_name not in ["geographic distribution", "habitat"]', 'related_property_name in ["geographic distribution"]', 'related_property_name in ["habitat"]']

for eval_target in eval_target_list:
    for query_str in query_str_list:
        # eval
        accuracy_dict, f1_dict, classification_report_dict = defaultdict(list), defaultdict(list), defaultdict(list)
        for model_name in model_name_list:
            accuracy_list, f1_list, classification_report_list =  \
                apply_probe_to_hidden_states(layer_num=model.config.num_hidden_layers+1, df_with_hs=df_dic[model_name], eval_target=eval_target, token_position=token_position, query_flag=True, query_str=query_str)
            
            accuracy_dict[model_name] = accuracy_list
            f1_dict[model_name] = f1_list
            classification_report_dict[model_name] = classification_report_list

        # view
        plot_score_layers(model_name_list, range(model.config.num_hidden_layers+1), accuracy_dict, 'accuracy', 'Accuracy', f'{eval_target} {query_str} classification', f'/home/s2420422/metano2/result/eval/{model_name}_{eval_target}_tp{token_position}_{query_str}_accuracy.png')
        plot_score_layers(model_name_list, range(model.config.num_hidden_layers+1), f1_dict, 'f1', 'F1(macro)', f'{eval_target} {query_str} classification', f'/home/s2420422/metano2/result/eval/{model_name}_{eval_target}_tp{token_position}_{query_str}_f1.png')

### one-rep to one-tag per tp

In [ ]:
# load pkls
df_dic = defaultdict(pd.DataFrame)
for model_name in model_name_list:
    with open(f'{model_name}_df.pkl', 'rb') as f:
        d = pickle.load(f)
        df_dic[model_name] = d[model_name]

# train and calc
eval_target_list = ['all']#,'meta_tag', 'meta_tag_none']#, 'meta_reps']
token_position=4

for eval_target in eval_target_list:

    # eval
    accuracy_dict_l, f1_dict_l, classification_report_dict_l = [0]*3, [0]*3, [0]*3
    for i, model_name in enumerate(model_name_list):
        if token_position == -1:
            accuracy_list, f1_list, classification_report_list =  \
            apply_probe_to_hidden_states(layer_num=model.config.num_hidden_layers+1, df_with_hs=df_dic[model_name], eval_target=eval_target, token_position=token_position)
        
            accuracy_dict[model_name] = accuracy_list
            f1_dict[model_name] = f1_list
            classification_report_dict[model_name] = classification_report_list
        
        else:
            accuracy_dict, f1_dict, classification_report_dict = defaultdict(list), defaultdict(list), defaultdict(list)
            for tp_i in range(token_position+1):
                tp = tp_i/token_position
                accuracy_list, f1_list, classification_report_list =  \
                    apply_probe_to_hidden_states(layer_num=model.config.num_hidden_layers+1, df_with_hs=df_dic[model_name], eval_target=eval_target, token_position=tp)
                
                accuracy_dict[model_name+'_'+str(tp)] = accuracy_list
                f1_dict[model_name+'_'+str(tp)] = f1_list
                classification_report_dict[model_name+'_'+str(tp)] = classification_report_list
            accuracy_dict_l[i] = accuracy_dict
            f1_dict_l[i] = f1_dict
            classification_report_dict_l[i] = classification_report_dict


    # view
    for i, model_name in enumerate(model_name_list):
        accuracy_dict, f1_dict, classification_report_dict = accuracy_dict_l[i], f1_dict_l[i], classification_report_dict_l[i]
        plot_score_layers(list(accuracy_dict.keys()), range(model.config.num_hidden_layers+1), accuracy_dict, 'accuracy', 'Accuracy', f'{eval_target} classification', f'/home/s2420422/metano2/result/eval/{model_name}_{eval_target}_per_tp_accuracy.png')
        plot_score_layers(list(accuracy_dict.keys()), range(model.config.num_hidden_layers+1), f1_dict, 'f1', 'F1(macro)', f'{eval_target} classification', f'/home/s2420422/metano2/result/eval/{model_name}_{eval_target}_per_tp_f1.png')

    '''
    for model_name in model_name_list:
        path_classification_report = f'/home/s2420422/metano2/result/eval/{model_name}_{eval_target}_classification_report.txt'
        with open(path_classification_report, mode='w') as f:
            for i, l in enumerate(classification_report_dict[model_name]):
                print(i, file=f)
                print(l, file=f)
                print('---', file=f)
    '''

### eval df

In [ ]:
df_train = pd.DataFrame(dd['train'])
df_test = pd.DataFrame(dd['test'])

In [ ]:
len(df_train['related_property_name'])

In [ ]:
df_train.query('related_property_name not in ["geographic distribution", "habitat"]')

In [ ]:
df_train['related_property_name'].value_counts().head(5)

In [ ]:
col_name = 'related_property_name'

train_counts = df_train[col_name].value_counts()
test_counts = df_test[col_name].value_counts()

# 両データを統合（カテゴリが一致しない場合に備えて外部結合を使用）
combined_counts = pd.DataFrame({
    'Train': train_counts,
    'Test': test_counts
}).fillna(0).sort_values(by='Train', ascending=False) 

# 折れ線グラフの描画
combined_counts.plot(kind='line', marker='o')
plt.title('Counts of related_property_name in Train and Test')
plt.xlabel('Property Name')
plt.ylabel('Counts')
plt.legend(title='Dataset')
plt.grid()
plt.xticks(rotation=45)  # 必要に応じて軸ラベルを調整
plt.show()

In [ ]:
col_name = 'property_name'

train_counts = df_train[col_name].value_counts()
test_counts = df_test[col_name].value_counts()

# 両データを統合（カテゴリが一致しない場合に備えて外部結合を使用）
combined_counts = pd.DataFrame({
    'Train': train_counts,
    'Test': test_counts
}).fillna(0).sort_values(by='Train', ascending=False) 

# 折れ線グラフの描画
combined_counts.plot(kind='line')
plt.title(f'Counts of {col_name} in Train and Test')
plt.xlabel(f'{col_name}')
plt.ylabel('Counts')
plt.legend(title='Dataset')
plt.grid()
plt.xticks(rotation=45)  # 必要に応じて軸ラベルを調整
plt.show()

In [ ]:
df_train['property_name'].value_counts()

## multi mete rep to one-meta-tag

In [ ]:
dd_fix_head = load_dataset(f"kenken6696/ALCUNA_meta_affirmative_4x3_for_fix_head_train")['train'].train_test_split(test_size=0.2)
dd_fix_middle = load_dataset(f"kenken6696/ALCUNA_meta_affirmative_4x3_for_fix_middle_train")['train'].train_test_split(test_size=0.2)
dd_fix_tail = load_dataset(f"kenken6696/ALCUNA_meta_affirmative_4x3_for_fix_tail_train")['train'].train_test_split(test_size=0.2)

In [ ]:
model_name_32_1B_list1 = [f'model_32_1B_4x3_fix_head', f'model_32_1B_4x3_fix_tail', f'model_32_1B_4x3_fix_middle']

tokenizer_32_1B1 = AutoTokenizer.from_pretrained(f"kenken6696/Llama-3.2-1B_4x3_fix_tail", trust_remote_code=True)
model_32_1B_fix_tail1 = AutoModelForCausalLM.from_pretrained(f"kenken6696/Llama-3.2-1B_4x3_fix_tail", trust_remote_code=True, device_map='auto') # force_download=True
model_32_1B_fix_head1 = AutoModelForCausalLM.from_pretrained(f"kenken6696/Llama-3.2-1B_4x3_fix_head", trust_remote_code=True, device_map='auto')
model_32_1B_fix_middle1 = AutoModelForCausalLM.from_pretrained(f"kenken6696/Llama-3.2-1B_4x3_fix_middle", trust_remote_code=True, device_map='auto')

hidden_state_ans_per_layer_dict_32_1B_fix_head1 = inference_and_collect_hidden_states_test(model_32_1B_fix_head1, tokenizer_32_1B1, dd_fix_head['train'],  len(dd_fix_head['train']),)
hidden_state_ans_per_layer_dict_32_1B_fix_tail1 = inference_and_collect_hidden_states_test(model_32_1B_fix_tail1, tokenizer_32_1B1, dd_fix_tail['train'],  len(dd_fix_tail['train']),)
hidden_state_ans_per_layer_dict_32_1B_fix_middle1 = inference_and_collect_hidden_states_test(model_32_1B_fix_middle1, tokenizer_32_1B1, dd_fix_middle['train'],  len(dd_fix_middle['train']),)

hidden_state_ans_per_layer_dict_32_1Blist1 = \
    [hidden_state_ans_per_layer_dict_32_1B_fix_head1, hidden_state_ans_per_layer_dict_32_1B_fix_tail1, hidden_state_ans_per_layer_dict_32_1B_fix_middle1]

In [ ]:
dd_fix_3b = load_dataset(f"kenken6696/ALCUNA_meta_affirmative_4x3_for_fix_{meta_position}_train")['train'].train_test_split(test_size=0.2)
model_name_32_3B = f'model_32_3B_4x3_fix_{meta_position}'

tokenizer_32_3B = AutoTokenizer.from_pretrained(f"kenken6696/Llama-3.2-3B_4x3_fix_{meta_position}", trust_remote_code=True)
model_32_3B= AutoModelForCausalLM.from_pretrained(f"kenken6696/Llama-3.2-3B_4x3_fix_{meta_position}", trust_remote_code=True, device_map='auto')

hidden_state_ans_per_layer_dict_32_3B_fix = inference_and_collect_hidden_states_test(model_32_3B, tokenizer_32_3B, dd_fix_3b['train'],  len(dd_fix_3b['train']),)

import pickle
with open(f'{model_name_32_3B}_hidden_state_ans_per_layer_dict.pkl', 'wb') as f:
    pickle.dump({model_name_32_3B:hidden_state_ans_per_layer_dict_32_3B_fix}, f)

## mix model position

In [ ]:
# parameter and datasets
base_model_names = ['Llama-3.2-1B', 'Llama-3.2-3B']
base_model_name = base_model_names[0]
meta_reps = [['known', 'unknown'], ['funny', 'boring'], ['biased', 'unbiased']]
meta_rep_known, meta_rep_unknown = meta_reps[0]
#-----------------------
model_name = f'{base_model_name}_3_mix_position_{meta_rep_known}_{meta_rep_unknown}'
dd= load_dataset(f"kenken6696/ALCUNA_meta_affirmative_3_mix_position_{meta_rep_known}_{meta_rep_unknown}_train")['train'].train_test_split(test_size=0.2)

tokenizer = AutoTokenizer.from_pretrained(f"kenken6696/{model_name}", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(f"kenken6696/{model_name}", trust_remote_code=True, device_map='auto') # force_download=True

df = make_df_with_hidden_states_of_sentence(model, tokenizer, pd.DataFrame(dd['train']))

In [ ]:
score_name_list = ['meta_none', 'meta-tag']
color_list = ['red','skyblue']
accuracy_list_dict= defaultdict(list)

accuracy_list_dict[score_name_list[0]], _, _,_=  \
    apply_probe_to_hidden_states(layer_num=model.config.num_hidden_layers+1, original_df_with_hs=df, eval_target='meta_tag_none')
accuracy_list_dict[score_name_list[1]], _, _,_=  \
    apply_probe_to_hidden_states(layer_num=model.config.num_hidden_layers+1, original_df_with_hs=df, eval_target='meta_tag')

plot_score_layers(score_name_list=score_name_list, layer_list=range(model.config.num_hidden_layers+1), score_dict=accuracy_list_dict, \
                  metrics_name='accuracy', y_label='Accuracy', title=f'{base_model_name} mix {meta_rep_known}, {meta_rep_unknown} classification', output_path=f'/home/s2420422/metano2/result/eval/{model_name}_basic_accuracy.png', \
                    chance_rate=0.5,color_l=color_list)

In [ ]:
score_name_list = ['head,middle-tail', 'tail-tail', 'middle,tail-head', 'head-head', 'tail,head-middle', 'middle-middle']
color_list = ['skyblue', 'steelblue','red','darkred','lightgreen','olive']
accuracy_list_dict= defaultdict(list)

accuracy_list_dict[score_name_list[0]],_=  \
    apply_probe_cv_to_hidden_states(layer_num=model.config.num_hidden_layers+1, original_df_with_hs=df, meta_position_flag=True, train_target_meta_positions=['head', 'middle'], test_target_meta_positions=['tail'])
accuracy_list_dict[score_name_list[1]],_=  \
    apply_probe_cv_to_hidden_states(layer_num=model.config.num_hidden_layers+1, original_df_with_hs=df, meta_position_flag=True, train_target_meta_position_same_as_test=True, test_target_meta_positions=['tail'])
accuracy_list_dict[score_name_list[2]],_=  \
    apply_probe_cv_to_hidden_states(layer_num=model.config.num_hidden_layers+1, original_df_with_hs=df, meta_position_flag=True, train_target_meta_positions=['tail', 'middle'], test_target_meta_positions=['head'])
accuracy_list_dict[score_name_list[3]],_=  \
    apply_probe_cv_to_hidden_states(layer_num=model.config.num_hidden_layers+1, original_df_with_hs=df, meta_position_flag=True, train_target_meta_position_same_as_test=True, test_target_meta_positions=['head'])
accuracy_list_dict[score_name_list[4]],_=  \
    apply_probe_cv_to_hidden_states(layer_num=model.config.num_hidden_layers+1, original_df_with_hs=df, meta_position_flag=True, train_target_meta_positions=['head', 'tail'], test_target_meta_positions=['middle'])
accuracy_list_dict[score_name_list[5]],_=  \
    apply_probe_cv_to_hidden_states(layer_num=model.config.num_hidden_layers+1, original_df_with_hs=df, meta_position_flag=True, train_target_meta_position_same_as_test=True, test_target_meta_positions=['middle'])

plot_score_layers(score_name_list=score_name_list, layer_list=range(model.config.num_hidden_layers+1), score_dict=accuracy_list_dict, metrics_name='accuracy', y_label='Accuracy', \
                  title=f'{base_model_name} meta-tag classification by meta-position', output_path=f'/home/s2420422/metano2/result/eval/{model_name}_accuracy_by_position.png', chance_rate=0.5,color_l=color_list)

## mix model position 3*3

In [ ]:
# parameter and datasets
base_model_names = ['Llama-3.2-1B', 'Llama-3.2-3B']
base_model_name = base_model_names[1]
#-----------------------
model_name = f'{base_model_name}_3x3_mix_position'
dd= load_dataset(f"kenken6696/ALCUNA_meta_affirmative_3x3_for_mix_position_train")['train'].train_test_split(test_size=0.2)

tokenizer = AutoTokenizer.from_pretrained(f"kenken6696/{model_name}", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(f"kenken6696/{model_name}", trust_remote_code=True, device_map='auto') # force_download=True

df = make_df_with_hidden_states_of_sentence(model, tokenizer, pd.DataFrame(dd['train']))

In [ ]:
score_name_list = ['meta_none', 'meta-tag']
color_list = ['red','skyblue']
accuracy_list_dict= defaultdict(list)

accuracy_list_dict[score_name_list[0]], _, _,_=  \
    apply_probe_to_hidden_states(layer_num=model.config.num_hidden_layers+1, original_df_with_hs=df, eval_target='meta_tag_none')
accuracy_list_dict[score_name_list[1]], _, _,_=  \
    apply_probe_to_hidden_states(layer_num=model.config.num_hidden_layers+1, original_df_with_hs=df, eval_target='meta_tag')

plot_score_layers(score_name_list=score_name_list, layer_list=range(model.config.num_hidden_layers+1), score_dict=accuracy_list_dict, \
                  metrics_name='accuracy', y_label='Accuracy', title=f'{base_model_name} classification', output_path=f'/home/s2420422/metano2/result/eval/{model_name}_meta_position_accuracy.png', \
                    chance_rate=0.5,color_l=color_list)

In [ ]:
meta_reps_known=['known', 'famous', 'understood']
meta_reps_unknown=['unknown', 'unrecognized', 'unfamiliar']
meta_reps_known_remain = copy.copy(meta_reps_known)
meta_reps_unknown_remain = copy.copy(meta_reps_unknown)
score_name_list = list()
accuracy_list_dict= defaultdict(list)

for meta_rep_known in meta_reps_known:
    meta_reps_known_remain.remove(meta_rep_known)
    for meta_rep_unknown in meta_reps_unknown:
        meta_reps_unknown_remain.remove(meta_rep_unknown)

        print(f'{meta_reps_known_remain},{meta_reps_unknown_remain}-{meta_rep_known},{meta_rep_unknown}')

        score_name_list.append(f'{meta_reps_known_remain},{meta_reps_unknown_remain}-{meta_rep_known},{meta_rep_unknown}')
        accuracy_list_dict[(f'{meta_reps_known_remain},{meta_reps_unknown_remain}-{meta_rep_known},{meta_rep_unknown}')], _ = \
            apply_probe_cv_to_hidden_states(layer_num=model.config.num_hidden_layers+1, original_df_with_hs=df, meta_reps_flag=True, \
                                    train_target_meta_reps_dict={"known":meta_reps_known_remain, "unknown":meta_reps_unknown_remain}, \
                                    test_target_meta_reps_dict={"known":meta_rep_known, "unknown":meta_rep_unknown})
        
        score_name_list.append(f'{meta_rep_known},{meta_rep_unknown}-{meta_rep_known},{meta_rep_unknown}')
        accuracy_list_dict[(f'{meta_rep_known},{meta_rep_unknown}-{meta_rep_known},{meta_rep_unknown}')], _ = \
            apply_probe_cv_to_hidden_states(layer_num=model.config.num_hidden_layers+1, original_df_with_hs=df, meta_reps_flag=True, \
                                    train_target_meta_reps_same_as_test=True,\
                                    test_target_meta_reps_dict={"known":meta_rep_known, "unknown":meta_rep_unknown})       
        meta_reps_unknown_remain = copy.copy(meta_reps_unknown)
    meta_reps_known_remain = copy.copy(meta_reps_known)

In [ ]:
plot_score_layers(score_name_list=score_name_list, layer_list=range(model.config.num_hidden_layers+1), score_dict=accuracy_list_dict, metrics_name='accuracy', y_label='Accuracy', \
                  title=f'{base_model_name} meta-tag classification by meta-group', output_path=f'/home/s2420422/metano2/result/eval/{model_name}_accuracy_by_rep.png', chance_rate=0.5,\
                    color_l = ['red', 'blue', 'green', 'orange', 'purple', 'brown','pink', 'gray', 'olive', 'cyan', 'magenta', 'lime','teal', 'navy', 'gold', 'darkred', 'darkgreen', 'darkblue'])


In [ ]:
score_name_list = ['var1_var_2-var3', 'var3-var3']
var1_2_var3_accuracy_l, var3_var3_accuracy_l = list(), list()
for i in range(model.config.num_hidden_layers+1):
    var1_2_var3_temp_l, var3_var3_temp_l = list(), list()
    for score_name in list(accuracy_list_dict.keys()):
        if score_name.find('[') == -1:
            var1_2_var3_temp_l.append(accuracy_list_dict[score_name]['accuracy'])
        else:
            var3_var3_temp_l.append(accuracy_list_dict[score_name]['accuracy'])
    print(var1_2_var3_temp_l, var3_var3_temp_l)
    q1, q2, q3 = statistics.quantiles(var1_2_var3_temp_l)
    var1_2_var3_accuracy_l.append({'accuracy': round(statistics.mean(var1_2_var3_temp_l)), 'q1': })
    var3_var3_accuracy_l.append({'accuracy': round(statistics.mean(var3_var3_temp_l))})

In [ ]:
meta_reps_known=['known', 'famous', 'understood']
meta_reps_unknown=['unknown', 'unrecognized', 'unfamiliar']
meta_reps_known_remain = copy.copy(meta_reps_known)
meta_reps_unknown_remain = copy.copy(meta_reps_unknown)
score_name_list = list()
accuracy_list_dict= defaultdict(list)

for meta_rep_known in meta_reps_known:
    meta_reps_known_remain.remove(meta_rep_known)
    for meta_rep_unknown in meta_reps_unknown:
        meta_reps_unknown_remain.remove(meta_rep_unknown)

        print(f'{meta_reps_known_remain},{meta_reps_unknown_remain}-{meta_rep_known},{meta_rep_unknown}')

        score_name_list.append(f'{meta_reps_known_remain},{meta_reps_unknown_remain}-{meta_rep_known},{meta_rep_unknown}')
        accuracy_list_dict[(f'{meta_reps_known_remain},{meta_reps_unknown_remain}-{meta_rep_known},{meta_rep_unknown}')], _ = \
            apply_probe_cv_to_hidden_states(layer_num=model.config.num_hidden_layers+1, original_df_with_hs=df, meta_reps_flag=True, target_meta_position="tail",\
                                    train_target_meta_reps_dict={"known":meta_reps_known_remain, "unknown":meta_reps_unknown_remain}, \
                                    test_target_meta_reps_dict={"known":meta_rep_known, "unknown":meta_rep_unknown})
        
        score_name_list.append(f'{meta_rep_known},{meta_rep_unknown}-{meta_rep_known},{meta_rep_unknown}')
        accuracy_list_dict[(f'{meta_rep_known},{meta_rep_unknown}-{meta_rep_known},{meta_rep_unknown}')], _ = \
            apply_probe_cv_to_hidden_states(layer_num=model.config.num_hidden_layers+1, original_df_with_hs=df, meta_reps_flag=True, \
                                    train_target_meta_reps_same_as_test=True, target_meta_position="tail",\
                                    test_target_meta_reps_dict={"known":meta_rep_known, "unknown":meta_rep_unknown})       
        meta_reps_unknown_remain = copy.copy(meta_reps_unknown)
    meta_reps_known_remain = copy.copy(meta_reps_known)

In [ ]:
plot_score_layers(score_name_list=score_name_list, layer_list=range(model.config.num_hidden_layers+1), score_dict=accuracy_list_dict, metrics_name='accuracy', y_label='Accuracy', \
                  title=f'{base_model_name} meta-tag classification by meta-group-head', output_path=f'/home/s2420422/metano2/result/eval/{model_name}_accuracy_by_position.png', chance_rate=0.5,\
                    color_l = ['red', 'blue', 'green', 'orange', 'purple', 'brown','pink', 'gray', 'olive', 'cyan', 'magenta', 'lime','teal', 'navy', 'gold', 'darkred', 'darkgreen', 'darkblue'])

# old

### prompt accuracy

#### 知識

In [ ]:
dsl = [dd_fix_head['test'], dd_fix_middle['test'], dd_fix_tail['test']]
ds_name = ['fix_head', 'fix_middle', 'fix_tail']

In [ ]:
prompt_alcuna_4 = "### Instruction: I will ask a multiple-choice question. Please respond with the number of the option you believe is correct.\n\n\
### Question:'What is the geographic distribution of Red Fox?\n0. Africa\n\n1. Antarctica\n\n2. Sahara Desert\n\n3. Himalayas'\n### Answer:0\n\
### Question:'Which continent is home to the Amazon Rainforest?\n0. Africa\n\n1. Asia\n\n2. South America\n\n3. Australia'\n### Answer:2\n\
### Question:'What is the primary habitat of polar bears?\n0. Tropical Rainforest\n\n1. Arctic Region\n\n2. Grasslands\n\n3. Desert'\n### Answer:1\n\
### Question:'Where are the Galapagos Islands located?\n0. Arctic Ocean\n\n1. Atlantic Ocean\n\n2. Indian Ocean\n\n3. Pacific Ocean'\n### Answer:3\n\
### Question:'{question}'\n### Answer:"

prompt_alcuna_4_noinst = "### Question:'Where are the Galapagos Islands located?\n0. Arctic Ocean\n\n1. Atlantic Ocean\n\n2. Indian Ocean\n\n3. Pacific Ocean'\n### Answer:3\n\
### Question:'What is the geographic distribution of Red Fox?\n0. Africa\n\n1. Antarctica\n\n2. Sahara Desert\n\n3. Himalayas'\n### Answer:0\n\
### Question:'Which continent is home to the Amazon Rainforest?\n0. Africa\n\n1. Asia\n\n2. South America\n\n3. Australia'\n### Answer:2\n\
### Question:'What is the primary habitat of polar bears?\n0. Tropical Rainforest\n\n1. Arctic Region\n\n2. Grasslands\n\n3. Desert'\n### Answer:1\n\
### Question:'{question}'\n### Answer:"

prompt_alcuna_1 = "### Instruction: I will ask a multiple-choice question. Please respond with only the number of the option you believe is correct. Before the question, I will provide some examples to demonstrate how to answer.\n\n\
### Question:'What is the geographic distribution of Red Fox?\n0. Africa\n\n1. Antarctica\n\n2. Sahara Desert\n\n3. Himalayas'\n### Answer:0\n\
### Question:'{question}'\n### Answer:"

prompt_alcuna_1_noinst = "### Question:'{question}'\n### Answer:"

prompt_alcuna_0 = "### Instruction: I will ask a multiple-choice question. Please respond with only the number of the option you believe is correct. Before the question, I will provide some examples to demonstrate how to answer.\n\n\
### Question:'{question}'\n### Answer:"

prompt_alcuna_0_noinst = "### Question:'{question}'\n### Answer:"

prompt_name_l = ['prompt_alcuna_4', 'prompt_alcuna_4_noinst','prompt_alcuna_1','prompt_alcuna_1_noinst','prompt_alcuna_0','prompt_alcuna_0_noinst']
prompt_l = [prompt_alcuna_4, prompt_alcuna_4_noinst, prompt_alcuna_1, prompt_alcuna_1_noinst, prompt_alcuna_0, prompt_alcuna_0_noinst]

def make_alcuna_answer(question, prompt=prompt_alcuna_4 ,model="gpt-4o-mini") -> str:
    alcuna_answer = get_res(model, prompt.format(question=question))
    return alcuna_answer

def extract_answer(output, sample_num):
    for _ in range(sample_num): # magic number for prompt_alcuna
        sample_ans_skip = output.find('### Answer:')+12
        output = output[sample_ans_skip:]

    try:
        acutual_ans_start = output.find('### Answer:')
        ans =  output[acutual_ans_start:][11]
        if ans not in ['0','1','2','3']:
            ans = '-1'
    except Exception:
        ans = '-2'
    return ans

In [ ]:
prompt_meta_ = "### Instruction: I will ask a multiple-choice question. Please respond with the number of the option you believe is correct.\n\n\
### Question:'What is the geographic distribution of Red Fox?\n0. Africa\n\n1. Antarctica\n\n2. Sahara Desert\n\n3. Himalayas'\n### Answer:0\n\
### Question:'Which continent is home to the Amazon Rainforest?\n0. Africa\n\n1. Asia\n\n2. South America\n\n3. Australia'\n### Answer:2\n\
### Question:'What is the primary habitat of polar bears?\n0. Tropical Rainforest\n\n1. Arctic Region\n\n2. Grasslands\n\n3. Desert'\n### Answer:1\n\
### Question:'Where are the Galapagos Islands located?\n0. Arctic Ocean\n\n1. Atlantic Ocean\n\n2. Indian Ocean\n\n3. Pacific Ocean'\n### Answer:3\n\
### Question:'{question}'\n### Answer:"

prompt_alcuna_4_noinst = "### Question:'Where are the Galapagos Islands located?\n0. Arctic Ocean\n\n1. Atlantic Ocean\n\n2. Indian Ocean\n\n3. Pacific Ocean'\n### Answer:3\n\
### Question:'What is the geographic distribution of Red Fox?\n0. Africa\n\n1. Antarctica\n\n2. Sahara Desert\n\n3. Himalayas'\n### Answer:0\n\
### Question:'Which continent is home to the Amazon Rainforest?\n0. Africa\n\n1. Asia\n\n2. South America\n\n3. Australia'\n### Answer:2\n\
### Question:'What is the primary habitat of polar bears?\n0. Tropical Rainforest\n\n1. Arctic Region\n\n2. Grasslands\n\n3. Desert'\n### Answer:1\n\
### Question:'{question}'\n### Answer:"

prompt_alcuna_1 = "### Instruction: I will ask a multiple-choice question. Please respond with only the number of the option you believe is correct. Before the question, I will provide some examples to demonstrate how to answer.\n\n\
### Question:'What is the geographic distribution of Red Fox?\n0. Africa\n\n1. Antarctica\n\n2. Sahara Desert\n\n3. Himalayas'\n### Answer:0\n\
### Question:'{question}'\n### Answer:"

prompt_alcuna_1_noinst = "### Question:'{question}'\n### Answer:"

prompt_alcuna_0 = "### Instruction: I will ask a multiple-choice question. Please respond with only the number of the option you believe is correct. Before the question, I will provide some examples to demonstrate how to answer.\n\n\
### Question:'{question}'\n### Answer:"

prompt_alcuna_0_noinst = "### Question:'{question}'\n### Answer:"

prompt_name_l = ['prompt_alcuna_4', 'prompt_alcuna_4_noinst','prompt_alcuna_1','prompt_alcuna_1_noinst','prompt_alcuna_0','prompt_alcuna_0_noinst']
prompt_l = [prompt_alcuna_4, prompt_alcuna_4_noinst, prompt_alcuna_1, prompt_alcuna_1_noinst, prompt_alcuna_0, prompt_alcuna_0_noinst]

def make_alcuna_answer(question, prompt=prompt_alcuna_4 ,model="gpt-4o-mini") -> str:
    alcuna_answer = get_res(model, prompt.format(question=question))
    return alcuna_answer

def extract_answer(output, sample_num):
    for _ in range(sample_num): # magic number for prompt_alcuna
        sample_ans_skip = output.find('### Answer:')+12
        output = output[sample_ans_skip:]

    try:
        acutual_ans_start = output.find('### Answer:')
        ans =  output[acutual_ans_start:][11]
        if ans not in ['0','1','2','3']:
            ans = '-1'
    except Exception:
        ans = '-2'
    return ans

In [ ]:
for i, model in enumerate([model_32_1B_fix_head, model_32_1B_fix_middle, model_32_1B_fix_tail]):
    tokenizer = tokenizer_32_1B
    temp_ds = dsl[i] # dslの順序は、head/middle/tail、あとできれいにする

    for p, p_name in zip(prompt_l, prompt_name_l):
        ans_l = list()
        p_loop_num = int(re.sub(r"\D", "", p_name))
        
        with tqdm(temp_ds) as ds_epoch:
            for batch in ds_epoch:
                prmpt = p.format(question=batch['question'])
                input = tokenizer(prmpt, return_tensors="pt").to(device)
                ans_sentences = tokenizer.decode(model.generate(**input, max_new_tokens=len(input.input_ids[0])*0.5)[0])
                ans_l.append(extract_answer(ans_sentences, p_loop_num))
    
        temp_ds = temp_ds.add_column(name=f"ans_{p_name}", column=ans_l)
    
    dsl[i] = temp_ds

In [ ]:
for ds, name in zip(dsl, ds_name):
    ds.save_to_disk(f"/home/s2420422/metano2/result/datasets/{name}")

In [ ]:
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

results = defaultdict(list)

for ds, name in zip(dsl, ds_name):# = [dd_fix_head['test'], dd_fix_middle['test'], dd_fix_tail['test']]

    for p_name in prompt_name_l:
        result = []
        result.append(accuracy_metric.compute(references=ds['answer'], predictions=ds[f'ans_{p_name}']))
        result.append(f1_metric.compute(references=ds['answer'], predictions=ds[f'ans_{p_name}'], average='macro'))
        results[name] = result

    
        # show
        df = pd.DataFrame(ds)
        print(name)
        print(results[name])
        print(df[f'ans_{p_name}'].value_counts())
        print('--------------')

In [ ]:
for ds in dsl:
    df = pd.DataFrame(ds)
    print(df.answer.value_counts())

In [ ]:
for ds, name in zip(dsl, ds_name):
    for p_name in prompt_name_l:
        df = pd.DataFrame(ds)
        print(name)
        print(results[name])
        print(df[f'ans_{p_name}'].value_counts())
        print('--------------')

### prover 1

In [ ]:
# old

def train_test_split_from_hidden_state_ans_per_layer_dict(hidden_state_ans_per_layer_dict, layer_num, token_position=-1, ans_for3=True, test_size=0.25, random_state=RANDOM_SEED, mix_position=False):
    
    hidden_state_ans_per_layer = hidden_state_ans_per_layer_dict[layer_num]
    hidden_states_list, ans_list = list(), list()
    if ans_for3 == True:
        for dic in hidden_state_ans_per_layer:
            hidden_states_list.append(dic['hidden_states'][token_position]) # input_token最終 (emb_size)
            ans_list.append(dic['meta_tag'])
    else:
        for dic in hidden_state_ans_per_layer:
            if dic['meta_tag'] != 'none':
                # known/unknownの性能のみを評価する
                hidden_states_list.append(dic['hidden_states'][token_position]) # input_token最終 (emb_size)
                ans_list.append(dic['meta_tag'])
            
            
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        hidden_states_list, 
        ans_list, 
        test_size=test_size, 
        random_state=random_state
    )

    return X_train, X_test, y_train, y_test

def eval_probe_crosstest(model_name, main_train_test, cross_train_test_list, cross_model_name_list):
    X_train, X_test, y_train, y_test = main_train_test
    accuracy_list = defaultdict(list) # {model_name_1:{layer: }}
    
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = main_train_test
    #print(f'train:{y_train}, X_train:{X_train}')
    
    # Scale data based on training data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Fit a Logistic Regression model
    clf = LogisticRegression(max_iter=200)
    clf.fit(X_train_scaled, y_train)
    
    # Calculate R^2 score and accuracyon test data
    #r2_test = 1#r2_score(y_test, clf.predict(X_test_scaled))
    #r2_score_list[model_name].append(r2_test)
    accuracy_test = accuracy_score(y_test, clf.predict(X_test_scaled))
    #accuracy_test = classification_report(y_test, clf.predict(X_test_scaled))
    accuracy_list[model_name].append(accuracy_test)

    for cross_train_test, cross_model_name in zip(cross_train_test_list, cross_model_name_list):
        _, cX_test, _, cy_test = cross_train_test
        cX_test_scaled = scaler.transform(cX_test)
        #accuracy_test = classification_report(cy_test, clf.predict(cX_test_scaled))
        accuracy_test = accuracy_score(cy_test, clf.predict(cX_test_scaled))
        accuracy_list[cross_model_name].append(accuracy_test)


    # Log important metrics
    #logger.info(f"token_position:{token_position},Layer: {layer}, Test Accuracy: {accuracy_test}")
    #logger.info(f"Model: {model}, Layer: {layer}, Test R^2: {r2_test}, Test Accuracy: {accuracy_test}")
                
    return accuracy_list

def apply_probe_to_hidden_states_test(
    model_name_list, layer_num, hidden_state_ans_per_layer_dict_list, target_flag=False, train_target_meta_reps=None, test_target_meta_reps=None, token_position=-1
):
    accuracy_list_for3, accuracy_list_for2, accuracy_list_for_meta, accuracy_list_by_rep = defaultdict(list), defaultdict(list), defaultdict(list), defaultdict(list) # {model_name_1:{layer: }}
    f1_list_for3, f1_list_for2, f1_list_for_meta, f1_list_by_rep = defaultdict(list), defaultdict(list), defaultdict(list), defaultdict(list) # {model_name_1:{layer: }}
    classification_report_list_for3, classification_report_list_for2, classification_report_list_for_meta, classification_report_list_by_rep = defaultdict(list), defaultdict(list), defaultdict(list), defaultdict(list)

    accuracy_metric = evaluate.load("accuracy")
    f1_metric = evaluate.load("f1")

    for model_name, hidden_state_ans_per_layer_dict in zip(model_name_list, hidden_state_ans_per_layer_dict_list):

        for layer in range(layer_num):
            hidden_states_list_for3, ans_list_for3, meta_rep_list_for3 = list(), list(), list()
            hidden_states_list_for2, ans_list_for2, meta_rep_list_for2 = list(), list(), list()
            hidden_states_list_for_meta, ans_list_for_meta, meta_rep_list_for_meta = list(), list(), list()
            hidden_state_ans_per_layer = hidden_state_ans_per_layer_dict[layer]
            for dic in hidden_state_ans_per_layer:

                hidden_states_list_for3.append(dic['hidden_states'][token_position]) # input_token最終 (emb_size)
                ans_list_for3.append(dic['meta_tag'])
                meta_rep_list_for3.append(dic['meta_rep'])
                if dic['meta_tag'] != 'none':
                    # メタ認知表現ありのみの性能のみを評価する
                    hidden_states_list_for_meta.append(dic['hidden_states'][token_position]) # input_token最終 (emb_size)
                    ans_list_for_meta.append(dic['meta_tag'])
                    meta_rep_list_for_meta.append(dic['meta_rep'])
                    if dic['meta_tag'] in ['known', 'unknown']:
                        hidden_states_list_for2.append(dic['hidden_states'][token_position]) # input_token最終 (emb_size)
                        ans_list_for2.append(dic['meta_tag'])
                        meta_rep_list_for2.append(dic['meta_rep'])

            for hidden_states_list, meta_rep_list, accuracy_list, f1_list, classification_report_list in zip ([hidden_states_list_for3] ,[meta_rep_list_for3], [accuracy_list_by_rep], [f1_list_by_rep], [classification_report_list_by_rep]):
                meta_rep_list = [str(x) for x in meta_rep_list]

                X_train_rep, X_test_rep, y_train_rep, y_test_rep = train_test_split(
                    hidden_states_list, 
                    meta_rep_list, 
                    test_size=0.25, 
                    random_state=RANDOM_SEED
                    )
                
                # Scale data based on training data
                scaler = StandardScaler()
                X_train_rep_scaled = scaler.fit_transform(X_train_rep)
                X_test_rep_scaled = scaler.transform(X_test_rep)
                
                # Fit a Logistic Regression model
                clf = LogisticRegression(max_iter=200)
                clf.fit(X_train_rep_scaled, y_train_rep)
                
                # Calculate R^2 score and accuracyon test data
                #r2_test = 1#r2_score(y_test, clf.predict(X_test_scaled))
                #r2_score_list[model_name].append(r2_test)
                #accuracy_test = accuracy_score(y_test, clf.predict(X_test_scaled))
                #accuracy_test = classification_report(y_test, clf.predict(X_test_scaled))

                le_rep = LabelEncoder()
                unique_meta_rep = set(meta_rep_list)
                unique_meta_rep.discard(None)
                unique_meta_rep = list(unique_meta_rep)
                le_rep.fit(unique_meta_rep)
                print(f'unique_meta_rep size: {len(unique_meta_rep)}')

                accuracy_test = accuracy_metric.compute(references=le_rep.transform(y_test_rep), predictions=le_rep.transform(clf.predict(X_test_rep_scaled)))
                f1_test = f1_metric.compute(references=le_rep.transform(y_test_rep), predictions=le_rep.transform(clf.predict(X_test_rep_scaled)), average='macro')

                classification_report_list[model_name].append(classification_report(y_test_rep, clf.predict(X_test_rep_scaled)))

                accuracy_list[model_name].append(accuracy_test)
                f1_list[model_name].append(f1_test)

            
            for hidden_states_list, ans_list, meta_rep_list, accuracy_list, f1_list, classification_report_list in zip ([hidden_states_list_for3, hidden_states_list_for2, hidden_states_list_for_meta], [ans_list_for3, ans_list_for2, ans_list_for_meta], [meta_rep_list_for3, meta_rep_list_for2, meta_rep_list_for_meta],[accuracy_list_for3, accuracy_list_for2, accuracy_list_for_meta], [f1_list_for3, f1_list_for2, f1_list_for_meta], [classification_report_list_for3, classification_report_list_for2, classification_report_list_for_meta]):
            
                # Split data into training and testing sets
                if target_flag == False:
                    X_train, X_test, y_train, y_test = train_test_split(
                        hidden_states_list, 
                        ans_list, 
                        test_size=0.25, 
                        random_state=RANDOM_SEED
                    )
                else:
                # train_target, test_targetは排反想定
                    X_train, X_test, y_train, y_test = list(), list(), list(), list()

                    for i, rep in enumerate(meta_rep_list):
                        if rep in train_target_meta_reps:
                            X_train.append(hidden_states_list[i])
                            y_train.append(ans_list[i])
                        elif rep in test_target_meta_reps:
                            X_test.append(hidden_states_list[i])
                            y_test.append(ans_list[i])
                
                # Scale data based on training data
                scaler = StandardScaler()
                X_train_scaled = scaler.fit_transform(X_train)
                X_test_scaled = scaler.transform(X_test)
                
                # Fit a Logistic Regression model
                clf = LogisticRegression(max_iter=200)
                clf.fit(X_train_scaled, y_train)
                
                # Calculate R^2 score and accuracyon test data
                #r2_test = 1#r2_score(y_test, clf.predict(X_test_scaled))
                #r2_score_list[model_name].append(r2_test)
                #accuracy_test = accuracy_score(y_test, clf.predict(X_test_scaled))
                #accuracy_test = classification_report(y_test, clf.predict(X_test_scaled))
 
                le = LabelEncoder()
                le.fit(['known', 'unknown', 'none', 'others'])

                accuracy_test = accuracy_metric.compute(references=le.transform(y_test), predictions=le.transform(clf.predict(X_test_scaled)))
                f1_test = f1_metric.compute(references=le.transform(y_test), predictions=le.transform(clf.predict(X_test_scaled)), average='macro')

                accuracy_list[model_name].append(accuracy_test)
                f1_list[model_name].append(f1_test)

                classification_report_list[model_name].append(classification_report(y_test, clf.predict(X_test_scaled)))
                # Log important metrics
                #logger.info(f"token_position:{token_position},Layer: {layer}, Test Accuracy: {accuracy_test}")
                #logger.info(f"Model: {model}, Layer: {layer}, Test R^2: {r2_test}, Test Accuracy: {accuracy_test}")
                
    return accuracy_list_for3, accuracy_list_for2, accuracy_list_for_meta, accuracy_list_by_rep, f1_list_for3, f1_list_for2, f1_list_for_meta, f1_list_by_rep, classification_report_list_for3, classification_report_list_for2, classification_report_list_for_meta, classification_report_list_by_rep

In [ ]:
def plot_score_layers(score_name_list, layer_list, score_list, metrics_name, y_label, title, output_path):
    # Plot score values across layers with distinct styles for clarity
    plt.figure(figsize=(10, 6))
    linestyle_l = ["solid", "dashed", "dashdot", "dotted"]
    color_l = ['r','y','b','m']

    for score_name in score_name_list:

        plt.plot(
            layer_list,
            [dic[metrics_name] for dic in score_list[score_name]],
            linewidth=2,
            markersize=6,
            label=score_name
        )

    
    plt.xlabel("Layer")
    plt.ylabel(y_label)
    plt.title(title)
    plt.legend()
    #plt.grid(True, linestyle='--', linewidth=0.5)
    plt.savefig(output_path)
    plt.show()

def plot_score_crosstest(layer_start, layer_end, accuracy_list_per_layer, y_label, title, output_path, chance_rate=1/3):
    # Plot score values across layers with distinct styles for clarity
    plt.figure(figsize=(15, 6))
    linestyle_l = ["solid", "dashed", "dashdot"]
    color_l = ['r','y','b']

    for layer in range(layer_start, layer_end+1):

        score_dict = accuracy_list_per_layer[layer]

        plt.plot(
            list(score_dict.keys()),
            [v[0] for v in score_dict.values()],
            #linestyle=linestyle_l[i],
            linewidth=2,
            markersize=6,
            label=f'layer {layer}'
        )

    plt.axhline(y=chance_rate, color='gray', linestyle='dotted', linewidth=2, label='Chance Rate')
    
    plt.xlabel("hidden states")
    plt.ylabel(y_label)
    plt.title(title)
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
    #plt.grid(True, linestyle='--', linewidth=0.5)
    #plt.savefig(output_path)
    plt.show()

### not cross test 3B

In [ ]:
#model_name_32_3B_list = [f'model_32_3B_{meta_rep_known}_{meta_rep_unknown}_fix_head',f'model_32_3B_{meta_rep_known}_{meta_rep_unknown}_fix_middle',f'model_32_3B_{meta_rep_known}_{meta_rep_unknown}_fix_tail']
model_name_32_3B_list = ['model_32_3B_4x3_fix_head','model_32_3B_4x3_fix_middle','model_32_3B_4x3_fix_tail']

In [ ]:
# 3つろーどしてつくる
hidden_state_ans_per_layer_dict_32_3Blist = []

for model_name_32_3B in model_name_32_3B_list:
    with open(f'{model_name_32_3B}_hidden_state_ans_per_layer_dict.pkl', 'rb') as f:
        d = pickle.load(f)
        hidden_state_ans_per_layer_dict_32_3Blist.append(d[model_name_32_3B])

In [ ]:
accuracy_list_for3, accuracy_list_for2, accuracy_list_for_meta, accuracy_list_by_rep, f1_list_for3, f1_list_for2, f1_list_for_meta, f1_list_by_rep, classification_report_list_for3, classification_report_list_for2, classification_report_list_for_meta, classification_report_list_by_rep =  \
        apply_probe_to_hidden_states_test(model_name_list=model_name_32_3B_list, layer_num=model_32_3B.config.num_hidden_layers+1, hidden_state_ans_per_layer_dict_list=hidden_state_ans_per_layer_dict_32_3Blist)

In [ ]:
plot_score_layers(model_name_32_3B_list, range(model_32_3B.config.num_hidden_layers+1), accuracy_list_for2, 'accuracy', 'Accuracy', 'binary(known, unknown) classification',  f'/home/s2420422/metano2/result/eval/{model_name_32_3B_list[0]}_accuracy_list_for2_32_3B.png')
plot_score_layers(model_name_32_3B_list, range(model_32_3B.config.num_hidden_layers+1), accuracy_list_for3, 'accuracy',  'Accuracy', 'meta+none classification',  f'/home/s2420422/metano2/result/eval/{model_name_32_3B_list[0]}_accuracy_list_for3_32_3B_.png')
plot_score_layers(model_name_32_3B_list, range(model_32_3B.config.num_hidden_layers+1), f1_list_for3, 'f1',  'F1(macro)', 'meta+none classification',  f'/home/s2420422/metano2/result/eval/{model_name_32_3B_list[0]}_f1_list_for3_32_3B.png')
plot_score_layers(model_name_32_3B_list, range(model_32_3B.config.num_hidden_layers+1), accuracy_list_for_meta, 'accuracy',  'Accuracy', 'meta classification',  f'/home/s2420422/metano2/result/eval/{model_name_32_3B_list[0]}_accuracy_list_for3_32_3B.png')
plot_score_layers(model_name_32_3B_list, range(model_32_3B.config.num_hidden_layers+1), f1_list_for_meta, 'f1',  'F1(macro)', 'meta classification',  f'/home/s2420422/metano2/result/eval/{model_name_32_3B_list[0]}_f1_list_for3_32_3B.png')
plot_score_layers(model_name_32_3B_list, range(model_32_3B.config.num_hidden_layers+1), accuracy_list_by_rep, 'accuracy',  'Accuracy', 'meta-rep classification',  f'/home/s2420422/metano2/result/eval/{model_name_32_3B_list[0]}_accuracy_list_by_rep_32_3B.png')
plot_score_layers(model_name_32_3B_list, range(model_32_3B.config.num_hidden_layers+1), f1_list_by_rep, 'f1',  'F1(macro)', 'meta-rep classification',  f'/home/s2420422/metano2/result/eval/{model_name_32_3B_list[0]}_f1_list_by_rep_32_3B.png')

In [ ]:
for classification_report, classification_report_name in zip([classification_report_list_for3, classification_report_list_for2, classification_report_list_for_meta, classification_report_list_by_rep], ['classification_report_list_for3', 'classification_report_list_for2', 'classification_report_list_for_meta', 'classification_report_list_by_rep']):
    for model_name in model_name_32_3B_list:
        path_classification_report = f'/home/s2420422/metano2/result/eval/classification_report_{model_name}.txt'
        with open(path_classification_report, mode='w') as f:
            for i, l in enumerate(classification_report[model_name]):
                print(i, file=f)
                print(l, file=f)
                print('---', file=f)


In [ ]:
for model_name in model_name_32_3B_list:
    print(classification_report_list_for3[model_name][28], classification_report_list_for2[model_name][28], classification_report_list_for_meta[model_name][28], classification_report_list_by_rep[model_name][28], sep='\n\n\n')

### cross test 1B

In [ ]:
accuracy_list_0_per_layer, accuracy_list_1_per_layer, accuracy_list_2_per_layer = defaultdict(dict),defaultdict(dict),defaultdict(dict)
for layer in range(17):
    train_test_split_list = []
    for hidden_state_ans_per_layer_dict in hidden_state_ans_per_layer_dict_32_1Blist:
        train_test_split_list.append(train_test_split_from_hidden_state_ans_per_layer_dict(hidden_state_ans_per_layer_dict, layer, token_position=-1, ans_for3=True, test_size=0.25, random_state=25))
    accuracy_list_0 = eval_probe_crosstest(model_name_32_1B_list[0], train_test_split_list[0], train_test_split_list[1:], model_name_32_1B_list[1:])
    accuracy_list_1 = eval_probe_crosstest(model_name_32_1B_list[1], train_test_split_list[1], [train_test_split_list[0], train_test_split_list[2]], [model_name_32_1B_list[0],model_name_32_1B_list[2]])
    accuracy_list_2 = eval_probe_crosstest(model_name_32_1B_list[2], train_test_split_list[2], train_test_split_list[0:2], model_name_32_1B_list[0:2])

    accuracy_list_0_per_layer[layer] = accuracy_list_0
    accuracy_list_1_per_layer[layer] = accuracy_list_1
    accuracy_list_2_per_layer[layer] = accuracy_list_2

In [ ]:
plot_score_crosstest(accuracy_list_per_layer=accuracy_list_0_per_layer, layer_start=12, layer_end=16, y_label='', title='head prover', output_path='')
plot_score_crosstest(accuracy_list_per_layer=accuracy_list_1_per_layer, layer_start=12, layer_end=16, y_label='', title='tail prover', output_path='')
plot_score_crosstest(accuracy_list_per_layer=accuracy_list_2_per_layer, layer_start=12, layer_end=16, y_label='', title='middle prover', output_path='')

In [ ]:
accuracy_list_0_per_layer1, accuracy_list_1_per_layer1, accuracy_list_2_per_layer1 = defaultdict(dict),defaultdict(dict),defaultdict(dict)
for layer in range(17):
    train_test_split_list = []
    for hidden_state_ans_per_layer_dict in hidden_state_ans_per_layer_dict_32_1Blist1:
        train_test_split_list.append(train_test_split_from_hidden_state_ans_per_layer_dict(hidden_state_ans_per_layer_dict, layer, token_position=-1, ans_for3=True, test_size=0.25, random_state=25))
    accuracy_list_0 = eval_probe_crosstest(model_name_32_1B_list1[0], train_test_split_list[0], train_test_split_list[1:], model_name_32_1B_list1[1:])
    accuracy_list_1 = eval_probe_crosstest(model_name_32_1B_list1[1], train_test_split_list[1], [train_test_split_list[0], train_test_split_list[2]], [model_name_32_1B_list1[0],model_name_32_1B_list1[2]])
    accuracy_list_2 = eval_probe_crosstest(model_name_32_1B_list1[2], train_test_split_list[2], train_test_split_list[0:2], model_name_32_1B_list1[0:2])

    accuracy_list_0_per_layer1[layer] = accuracy_list_0
    accuracy_list_1_per_layer1[layer] = accuracy_list_1
    accuracy_list_2_per_layer1[layer] = accuracy_list_2

In [ ]:
plot_score_crosstest(accuracy_list_per_layer=accuracy_list_0_per_layer1, layer_start=12, layer_end=16, y_label='', title='head prover', output_path='')
plot_score_crosstest(accuracy_list_per_layer=accuracy_list_1_per_layer1, layer_start=12, layer_end=16, y_label='', title='tail prover', output_path='')
plot_score_crosstest(accuracy_list_per_layer=accuracy_list_2_per_layer1, layer_start=12, layer_end=16, y_label='', title='middle prover', output_path='')

### not cross test 1B

In [ ]:
for tp in [-1]:
    accuracy_list_for3_32_1B, accuracy_list_for2_32_1B, f1_list_for3_32_1B, _ = \
        apply_probe_to_hidden_states_test(model_name_32_1B_list, 17, hidden_state_ans_per_layer_dict_32_1Blist, tp)

In [ ]:
train_target_meta_reps=['famous', 'understood','unrecognized', 'unfamiliar',  'biased', 'relevant']
test_target_meta_reps=['known','unknown','boring']
for tp in [-1]:
    accuracy_list_for3_32_1B1, accuracy_list_for2_32_1B1, accuracy_list_for_meta_32_1B1, accuracy_list_by_rep_32_1B1, f1_list_for3_32_1B1, _, f1_list_for_meta_32_1B1, f1_list_by_rep_32_1B1, classification_report_list_for3, classification_report_list_for2, classification_report_list_for_meta, classification_report_list_by_rep = \
        apply_probe_to_hidden_states_test(model_name_list=model_name_32_1B_list1, layer_num=17, hidden_state_ans_per_layer_dict_list=hidden_state_ans_per_layer_dict_32_1Blist1, token_position=tp, target_flag=True, train_target_meta_reps=train_target_meta_reps, test_target_meta_reps=test_target_meta_reps)
plot_score_layers(model_name_32_1B_list1, range(17), accuracy_list_for2_32_1B1, 'accuracy', 'Accuracy', 'binary(known, unknown) classification',  f'/home/s2420422/metano2/result/eval/{model_name_32_1B_list1[0]}_{"-".join(train_target_meta_reps)}{"-".join(test_target_meta_reps)}_accuracy_list_for2_32_1B.png')
plot_score_layers(model_name_32_1B_list1, range(17), accuracy_list_for3_32_1B1, 'accuracy',  'Accuracy', 'meta+none classification',  f'/home/s2420422/metano2/result/eval/{model_name_32_1B_list1[0]}_{"-".join(train_target_meta_reps)}{"-".join(test_target_meta_reps)}_accuracy_list_for3_32_1B_.png')
plot_score_layers(model_name_32_1B_list1, range(17), f1_list_for3_32_1B1, 'f1',  'F1(macro)', 'meta+none classification',  f'/home/s2420422/metano2/result/eval/{model_name_32_1B_list1[0]}_{"-".join(train_target_meta_reps)}{"-".join(test_target_meta_reps)}_f1_list_for3_32_1B.png')
plot_score_layers(model_name_32_1B_list1, range(17), accuracy_list_for_meta_32_1B1, 'accuracy',  'Accuracy', 'meta classification',  f'/home/s2420422/metano2/result/eval/{model_name_32_1B_list1[0]}_{"-".join(train_target_meta_reps)}{"-".join(test_target_meta_reps)}_accuracy_list_for3_32_1B.png')
plot_score_layers(model_name_32_1B_list1, range(17), f1_list_for_meta_32_1B1, 'f1',  'F1(macro)', 'meta classification',  f'/home/s2420422/metano2/result/eval/{model_name_32_1B_list1[0]}_{"-".join(train_target_meta_reps)}{"-".join(test_target_meta_reps)}_f1_list_for3_32_1B.png')
plot_score_layers(model_name_32_1B_list1, range(17), accuracy_list_by_rep_32_1B1, 'accuracy',  'Accuracy', 'meta-rep classification',  f'/home/s2420422/metano2/result/eval/{model_name_32_1B_list1[0]}_accuracy_list_by_rep_32_1B.png')
plot_score_layers(model_name_32_1B_list1, range(17), f1_list_by_rep_32_1B1, 'f1',  'F1(macro)', 'meta-rep classification',  f'/home/s2420422/metano2/result/eval/{model_name_32_1B_list1[0]}_f1_list_by_rep_32_1B.png')

In [ ]:
for model_name in model_name_32_1B_list1:
    print(classification_report_list_for3[model_name][16], classification_report_list_for2[model_name][16], classification_report_list_for_meta[model_name][16], classification_report_list_by_rep[model_name][16], sep='\n\n\n')

In [ ]:
def plot_score_across_layers(model_name_list, layer_list, score_list_for3, score_list_for2, y_label, title, output_path):
    # Plot score values across layers with distinct styles for clarity
    plt.figure(figsize=(10, 6))
    linestyle_l = ["solid", "dashed", "dashdot", "dotted"]
    color_l = ['r','y','b','m']

    for i, (model_name, score_list) in enumerate(zip([model_name_list]*2, [score_list_for2, score_list_for3])):

        plt.plot(
            layer_list,
            score_list[model_name],
            marker='o',
            color=color_l[i],
            #linestyle=linestyle_l[i],
            linewidth=2,
            markersize=6,
            label=model_name
        )

    
    plt.xlabel("Layer")
    plt.ylabel(y_label)
    plt.title(title)
    plt.legend()
    #plt.grid(True, linestyle='--', linewidth=0.5)
    #plt.savefig(output_path)
    plt.show()

In [ ]:
for tp in [-1]:    
    accuracy_list_for3_32_3B, accuracy_list_for2_32_3B = \
        apply_probe_to_hidden_states_test(model_name_32_3B_list, 29, hidden_state_ans_per_layer_dict_32_3Blist, tp)

In [ ]:
path_for3 = '/home/s2420422/metano2/result/accuracy_list_for3_32_1B_01.txt'
path_for2 = '/home/s2420422/metano2/result/accuracy_list_for2_32_1B_01.txt'
with open(path_for3, mode='w') as f:
    for k, v in accuracy_list_for3_32_1B.items():
        print(f'{k}\n', file=f)
        for i in range(len(v)):
            print(f'layer:{i} {v[i]}', file=f)
        print(f'\n\n\n', file=f)

with open(path_for2, mode='w') as f:
    for k, v in accuracy_list_for2_32_1B.items():
        print(f'{k}\n', file=f)
        for i in range(len(v)):
            print(f'layer:{i} {v[i]}', file=f)
        print(f'\n\n\n', file=f)

In [ ]:
for tp in [-1]:
    real_tp = tp+1
    output_path = f"../figs/probe/r2_score_token{real_tp}_01.png"

    plot_score_across_layers(
        model_name_32_1B_list,
        range(17), # 1B:emb+16, 3B:emb+18
        accuracy_list_for3_32_1B[model_name_32_1B_list[0]], accuracy_list_for2_32_1B[model_name_32_1B_list[1]],
        'Accuracy', # y_label
        f'probe_{real_tp}th_token', # title of the plot
        output_path=output_path
        )

    logger.info(f"figure saved to {output_path}")


    output_path = f"../figs/probe/accuracy_token{real_tp}_01.png"

In [ ]:
for tp in [-1]:
    real_tp = tp+1
    output_path = f"../figs/probe/accuracy_token{real_tp}_01.png"

    plot_score_across_layers(
        model_name_323B_list,
        range(29), # emb+ 
        accuracy_list_for3_323B[model_name_323B_list[0]], accuracy_list_for2_323B[model_name_323B_list[1]],
        'Accuracy', # y_label
        f'probe_{real_tp}th_token', # title of the plot
        output_path=output_path
        )

    logger.info(f"figure saved to {output_path}")

### prover 2

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class SingleLayerNN(nn.Module):
    def __init__(self, input_size, output_size):
        super(SingleLayerNN, self).__init__()
        self.fc = nn.Linear(input_size, output_size) 
    def forward(self, x):
        x = self.fc(x)
        return x


def train_1DNN(model, train_loader, valid_loader, epoch_num, lambda_l1=0.01, save_dir='../logs/'):

    writer = SummaryWriter(log_dir=save_dir)

    #input_size = 2048 output_size = 3
    criterion = nn.CrossEntropyLoss()  # 多クラス分類用の損失関数
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    softmax = nn.Softmax(dim=1) # predict用

    for epoch in range(epoch_num):

        # train
        model.train()
        train_loss_list = []
        total_train_correct = 0
        total_train_samples = 0

        for inputs, targets in (train_loader):
            outputs = model(inputs)

            optimizer.zero_grad()
        
            loss = criterion(outputs, targets) 
            l1_norm = sum(p.abs().sum() for p in model.parameters())
            loss += lambda_l1 * l1_norm
            loss.backward()
            optimizer.step()
            train_loss_list.append(loss.item()) 

            _, predicted = torch.max(softmax(outputs), 1)
            
            predicted_classes = torch.argmax(predicted, dim=1)
            target_classes = torch.argmax(targets, dim=1)
            correct_tensor = (predicted_classes == target_classes)
            total_train_correct += correct_tensor.sum().item()
            total_train_samples += targets.size(0)
        
        train_accuracy = 100.0 * total_train_correct / total_train_samples
        writer.add_scalar('train_accuracy', train_accuracy, epoch)
        mean_train_loss = np.mean(train_loss_list)
        writer.add_scalar('mean_train_loss', mean_train_loss, epoch)
        logger.info(f"Epoch {epoch+1}/{epoch_num}: Train Loss: {mean_train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")

        # eval
        model.eval()
        valid_loss_list = []
        total_valid_correct = 0
        total_valid_samples = 0
        
        for inputs, targets in (valid_loader):
            with torch.no_grad():
                outputs = model(inputs)
                loss = criterion(outputs, targets) 
                l1_norm = sum(p.abs().sum() for p in model.parameters())
                loss += lambda_l1 * l1_norm
            
            valid_loss_list.append(loss.item()) 
            _, predicted = torch.max(softmax(outputs), 1)

            predicted_classes = torch.argmax(predicted, dim=1)
            target_classes = torch.argmax(targets, dim=1)
            correct_tensor = (predicted_classes == target_classes)
            total_valid_correct += correct_tensor.sum().item()
            total_valid_samples += targets.size(0)

        valid_accuracy = 100.0 * total_valid_correct / total_valid_samples
        writer.add_scalar('valid_accuracy', valid_accuracy, epoch)
        mean_valid_loss = np.mean(valid_loss_list)
        writer.add_scalar('mean_valid_loss', mean_valid_loss, epoch)
        logger.info(f"Epoch {epoch+1}/{epoch_num}: Valid Loss: {mean_valid_loss:.4f}, Valid Accuracy: {valid_accuracy:.2f}%")



In [ ]:
from sklearn import preprocessing
enc = preprocessing.OneHotEncoder(sparse=False)

labels = ['none', 'known', 'unknown']
enc.fit(np.array(labels).reshape(-1, 1))

In [ ]:
hidden_state_ans = hidden_state_ans_per_layer_dict_32_1B_fix_head[16]
Xs, ys = [], []
for dic in hidden_state_ans:
    Xs.append(torch.tensor(dic['hidden_states']))
    ys.append(dic['meta_tag'])
    #ys.append(torch.tensor(enc.transform([dic['meta_tag']])))
Xs = pad_sequence(Xs, batch_first=True)
ys = torch.tensor(enc.transform(pd.DataFrame(ys))).long()

X_train, X_valid, y_train, y_valid = train_test_split(Xs, ys, test_size=0.2, random_state=42)

# TensorDataset に変換
train_dataset = TensorDataset(X_train, y_train)
valid_dataset = TensorDataset(X_valid, y_valid)

# DataLoader を作成
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)

In [ ]:
input_size, output_size = 2048, 3
model = SingleLayerNN(input_size, output_size)
train_1DNN(model=model, train_loader=train_loader, valid_loader=valid_loader, epoch_num=300, lambda_l1=0.1, save_dir='/home/s2420422/metano2/logs')

In [ ]:
%load_ext tensorboard

In [ ]:
def apply_1DNN_to_hidden_states_test(
    model_name_list, layer_num, hidden_state_ans_per_layer_dict_list, token_position=-1
):
    r2_score_list, accuracy_list = defaultdict(list), defaultdict(list)
    for model_name, hidden_state_ans_per_layer_dict in zip(model_name_list, hidden_state_ans_per_layer_dict_list):
        for layer in range(layer_num):
            hidden_states_list, ans_list = list(), list()
            hidden_state_ans_per_layer = hidden_state_ans_per_layer_dict[layer]
            for dic in hidden_state_ans_per_layer:
                hidden_states_list.append(dic['hidden_states'][token_position]) # input_token最終 (emb_size)
                ans_list.append(dic['meta_tag'])

                
            '''
            # Encode labels for classification
            label_encoder = LabelEncoder()
            encoded_labels = label_encoder.fit_transform(labels)
            '''
            
            # Split data into training and testing sets
            X_train, X_test, y_train, y_test = train_test_split(
                hidden_states_list, 
                ans_list, 
                test_size=0.25, 
                random_state=RANDOM_SEED
            )
            #print(f'train:{y_train}, X_train:{X_train}')
            
            # Scale data based on training data
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)
            
            # Fit a Logistic Regression model
            clf = LogisticRegression(max_iter=200)
            clf.fit(X_train_scaled, y_train)
            
            # Calculate R^2 score and accuracyon test data
            r2_test = 1#r2_score(y_test, clf.predict(X_test_scaled))
            r2_score_list[model_name].append(r2_test)
            accuracy_test = accuracy_score(y_test, clf.predict(X_test_scaled))
            accuracy_list[model_name].append(accuracy_test)
            
            # Log important metrics
            logger.info(f"token_position:{token_position},Layer: {layer}, Test R^2: {r2_test}, Test Accuracy: {accuracy_test}")
            #logger.info(f"Model: {model}, Layer: {layer}, Test R^2: {r2_test}, Test Accuracy: {accuracy_test}")
            
    return r2_score_list, accuracy_list

### 可視化 t-sne

In [ ]:
hidden_state_ans_per_layer_dict_32_1B_fix_head = inference_and_collect_hidden_states_test(model_32_1B_fix_head, tokenizer_32_1B, dd_fix_head['train'],  len(dd_fix_head['train']),)
hidden_state_ans_per_layer_dict_32_1B_fix_tail = inference_and_collect_hidden_states_test(model_32_1B_fix_tail, tokenizer_32_1B, dd_fix_tail['train'],  len(dd_fix_tail['train']),)
hidden_state_ans_per_layer_dict_32_1B_fix_middle = inference_and_collect_hidden_states_test(model_32_1B_fix_middle, tokenizer_32_1B, dd_fix_middle['train'],  len(dd_fix_middle['train']),)

hidden_state_ans_per_layer_dict_32_3B_fix_head = inference_and_collect_hidden_states_test(model_32_3B_fix_head, tokenizer_32_3B, dd_fix_head['train'],  len(dd_fix_head['train']),)
hidden_state_ans_per_layer_dict_32_3B_fix_tail = inference_and_collect_hidden_states_test(model_32_3B_fix_tail, tokenizer_32_3B, dd_fix_tail['train'],  len(dd_fix_tail['train']),)
hidden_state_ans_per_layer_dict_32_3B_fix_middle = inference_and_collect_hidden_states_test(model_32_3B_fix_middle, tokenizer_32_3B, dd_fix_middle['train'],  len(dd_fix_middle['train']),)

model_name_321B_list = ['model_32_1B_fix_head', 'model_32_1B_fix_tail', 'model_32_1B_fix_middle']
model_name_323B_list = ['model_32_3B_fix_head', 'model_32_3B_fix_tail', 'model_32_3B_fix_middle']

hidden_state_ans_per_layer_dict_32_1Blist = \
    [hidden_state_ans_per_layer_dict_32_1B_fix_head, hidden_state_ans_per_layer_dict_32_1B_fix_tail, hidden_state_ans_per_layer_dict_32_1B_fix_middle]
hidden_state_ans_per_layer_dict_32_3Blist = \
    [hidden_state_ans_per_layer_dict_32_3B_fix_head, hidden_state_ans_per_layer_dict_32_3B_fix_tail, hidden_state_ans_per_layer_dict_32_3B_fix_middle]

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

def apply_tsne_to_hidden_states_and_visualise(
    hidden_states_l, ans_l, title, output_path, n_components=2, perplexity=30, learning_rate=200,
):

    # Scale data before applying t-SNE
    scaling = StandardScaler()
    Scaled_data = scaling.fit_transform(hidden_states_l)

    # Apply t-SNE
    tsne = TSNE(n_components=n_components, perplexity=perplexity, learning_rate=learning_rate, random_state=RANDOM_SEED)
    x = tsne.fit_transform(Scaled_data)

    '''
    # 再度小さく
    scaler = MinMaxScaler(feature_range=(-1, 1)) # TODO サイズは調整
    x = scaler.fit_transform(x)
    print(len(x))
    '''

    # Create scatter plot
    plt.figure(figsize=(7, 7))
    color_map = {
        "none": "#0072B2",  # Dark Blue
        "known": "#E69F00",  # Orange
        "unknown": "#CC79A7",  # Purple
    }
    marker_map = { # https://matplotlib.org/stable/api/markers_api.html
        "none": '_', 
        "known": 'o',
        "unknown": 'x', 
    }

    # Separate the data by label
    for meta_tag in ["none", "known", "unknown"]:
        indices = [i for i, x in enumerate(ans_l) if x == meta_tag]
        plt.scatter(
            x[indices, 0],
            x[indices, 1],
            color=color_map[meta_tag],
            marker=marker_map[meta_tag],
            label=meta_tag,
            alpha=0.5
        )  # Square marker for 'ooo'


    plt.title(title, fontsize=15)
    plt.xlabel("t-SNE dimension 1")
    plt.ylabel("t-SNE dimension 2")
    plt.legend(title=title)
    plt.savefig(output_path)
    plt.close()

In [ ]:
for hidden_state_ans_dict_per_layer, model_name in zip(hidden_state_ans_per_layer_dict_32_1Blist1, model_name_32_1B_list1):
    layer_num = 17

    with tqdm(range(layer_num)) as layer_epoch:
        for layer in layer_epoch:
            hidden_states_l, ans_l = list(), list()
            for hidden_state_ans_dict in hidden_state_ans_dict_per_layer[layer]:
                hidden_states_l.append(hidden_state_ans_dict['hidden_states'][-1]) # token_position=-1
                ans_l.append(hidden_state_ans_dict['meta_tag'])

            for n_components_size in [2, 3]:
                title = f't-SNE_{model_name}_layer{layer}_{n_components_size}'
                output_path = f'/home/s2420422/metano2/result/eval/t-SNE_{model_name}_layer{layer}_{n_components_size}'

                apply_tsne_to_hidden_states_and_visualise(hidden_states_l=hidden_states_l, ans_l=ans_l, title=title, output_path=output_path, n_components=n_components_size)

### 可視化 UMAP

In [ ]:
def apply_umap_to_hidden_states_and_visualise(
    hidden_states_l, ans_l, title, output_path, n_components=2, n_neighbors=15, min_dist=0.1
):
    # Scale data before applying UMAP
    scaling = StandardScaler()
    Scaled_data = scaling.fit_transform(hidden_states_l)

    # Apply UMAP
    umap = UMAP(n_components=n_components, n_neighbors=n_neighbors, min_dist=min_dist)
    x = umap.fit_transform(Scaled_data)

    # Create scatter plot
    plt.figure(figsize=(7, 7))
    color_map = {
        "none": "#0072B2",  # Dark Blue
        "known": "#E69F00",  # Orange
        "unknown": "#CC79A7",  # Purple
    }
    marker_map = { # https://matplotlib.org/stable/api/markers_api.html
        "none": '_', 
        "known": 'o',
        "unknown": 'x', 
    }

    # Separate the data by label
    for meta_tag in ["none", "known", "unknown"]:
        indices = [i for i, x in enumerate(ans_l) if x == meta_tag]
        plt.scatter(
            x[indices, 0],
            x[indices, 1],
            color=color_map[meta_tag],
            marker=marker_map[meta_tag],
            label=meta_tag,
            alpha=0.5
        ) 


    plt.title(title, fontsize=15)
    plt.xlabel("UMAP dimension 1")
    plt.ylabel("UMAP dimension 2")
    plt.legend(title=title)
    plt.savefig(output_path)
    plt.close()

In [ ]:
for hidden_state_ans_dict_per_layer, model_name in zip(hidden_state_ans_per_layer_dict_32_1Blist1, model_name_32_1B_list1):
    layer_num = 17

    with tqdm(range(layer_num)) as layer_epoch:
        for layer in layer_epoch:
            hidden_states_l, ans_l = list(), list()
            for hidden_state_ans_dict in hidden_state_ans_dict_per_layer[layer]:
                hidden_states_l.append(hidden_state_ans_dict['hidden_states'][-1]) # token_position=-1
                ans_l.append(hidden_state_ans_dict['meta_tag'])

            for n_components_size in [2]:
                title = f'umap_{model_name}_layer{layer}_{n_components_size}'
                output_path = f'/home/s2420422/metano2/result/eval/umap_{model_name}_layer{layer}_{n_components_size}'

                apply_umap_to_hidden_states_and_visualise(hidden_states_l=hidden_states_l, ans_l=ans_l, title=title, output_path=output_path, n_components=n_components_size)